In [ ]:
import glob, scipy, sklearn
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV

pd.set_option('display.max_columns', 100)
rand_seed = 10
#10, 14

In [ ]:
from matplotlib import pyplot as plt

def new_fig(figsize=(8.9, 5)):
    plt.style.use('classic')
    plt.close()
    fig = plt.figure(figsize=figsize)
    ax1 = fig.add_subplot(111)
    for axis in ['top', 'bottom', 'left', 'right']:
        ax1.spines[axis].set_linewidth(3)
    ax1.yaxis.grid(linestyle='--')
    for tick in ax1.xaxis.get_major_ticks():
        tick.label.set_fontsize(18)
    for tick in ax1.yaxis.get_major_ticks():
        tick.label.set_fontsize(18)
    for line in ax1.get_xticklines() + ax1.get_yticklines():
        line.set_markersize(8)
    return ax1

plt.style.use('classic')

In [ ]:
import random

core_num = 36
lv_node_num = 32
hs_node_num = 32

def df2csv(df, filename):
    f = open(filename, "w")
    f.write(df.to_csv(sep='\t', header=False, index=False))
    f.close()

In [ ]:
'''
random.seed(2019)
def gen_lv_smpl(lv_smpl_num, lv_smpl_filename, lmp_smpl_filename, voro_smpl_filename):
    lv_smpls = set([])
    while (len(lv_smpls) < lv_smpl_num):
        lmp_nproc = random.randint(2, (core_num - 1) * (lv_node_num - 1))
        lmp_ppw = random.randint(1, core_num - 1)
        lmp_nthread = random.randint(1, 4)
        lmp_io_step = random.randint(1, 8) * 50
        voro_nproc = random.randint(2, (core_num - 1) * (lv_node_num - 1))
        voro_ppw = random.randint(1, core_num - 1)
        voro_nthread = random.randint(1, 4)
        
        if (lmp_nproc >= lmp_ppw and voro_nproc >= voro_ppw):
            if (lmp_nproc % lmp_ppw == 0 and voro_nproc % voro_ppw == 0):
                nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw
            elif (lmp_nproc % lmp_ppw == 0 or voro_nproc % voro_ppw == 0):
                nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 1
            else:
                nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 2
            if (nodes <= lv_node_num):
                lv_smpls.add((lmp_nproc, lmp_ppw, lmp_nthread, lmp_io_step, voro_nproc, voro_ppw, voro_nthread))

    lv_smpls_df = pd.DataFrame(data = list(lv_smpls), \
                               columns=('lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step', \
                                        'voro_nproc', 'voro_ppw', 'voro_nthread'))
    print lv_smpls_df.head(10)
    df2csv(lv_smpls_df, lv_smpl_filename)
    
    lmp_smpls_df = pd.DataFrame(np.c_[lv_smpls_df.values[:, 0:4]], \
                                columns=('lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step'))
    lmp_smpls_df = lmp_smpls_df.drop_duplicates().reset_index(drop=True)
    print lmp_smpls_df.head(3)
    print lmp_smpls_df.shape[0]
    df2csv(lmp_smpls_df, lmp_smpl_filename)
    
    voro_smpls_df = pd.DataFrame(np.c_[lv_smpls_df.values[:, 4:7], lv_smpls_df.values[:, 3]], \
                                 columns=('voro_nproc', 'voro_ppw', 'voro_nthread', 'lmp_io_step'))
    voro_smpls_df = voro_smpls_df.drop_duplicates().reset_index(drop=True)
    print voro_smpls_df.head(3)
    print voro_smpls_df.shape[0]
    df2csv(voro_smpls_df, voro_smpl_filename)

gen_lv_smpl(10000, "conf_lv_smpls.csv", "conf_l_smpls.csv", "conf_v_smpls.csv")
'''

In [ ]:
'''
random.seed(2020)
def gen_lv_smpl_in(lv_smpl_num, lv_smpl_filename):
    lv_smpls = set([])
    while (len(lv_smpls) < lv_smpl_num):
        lmp_l2s = 16000 / (2 ** random.randint(0, 5))
        lmp_solid = 20000 / (2 ** random.randint(0, 5))
        lmp_nproc = random.randint(2, (core_num - 1) * (lv_node_num - 1))
        lmp_ppw = random.randint(1, core_num - 1)
        lmp_nthread = random.randint(1, 4)
        lmp_io_step = random.randint(1, 8) * 50
        voro_nproc = random.randint(2, (core_num - 1) * (lv_node_num - 1))
        voro_ppw = random.randint(1, core_num - 1)
        voro_nthread = random.randint(1, 4)
        
        if (lmp_nproc >= lmp_ppw and voro_nproc >= voro_ppw):
            if (lmp_nproc % lmp_ppw == 0 and voro_nproc % voro_ppw == 0):
                nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw
            elif (lmp_nproc % lmp_ppw == 0 or voro_nproc % voro_ppw == 0):
                nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 1
            else:
                nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 2
            if (nodes <= lv_node_num):
                lv_smpls.add((lmp_l2s, lmp_solid, lmp_nproc, lmp_ppw, lmp_nthread, lmp_io_step, voro_nproc, voro_ppw, voro_nthread))

    lv_smpls_df = pd.DataFrame(data = list(lv_smpls), \
                               columns=('lmp_l2s', 'lmp_solid', 'lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step', \
                                        'voro_nproc', 'voro_ppw', 'voro_nthread'))
    print lv_smpls_df.head(10)
    df2csv(lv_smpls_df, lv_smpl_filename)

gen_lv_smpl_in(10000, "conf_lv_in_smpls.csv")
'''

In [ ]:
'''
random.seed(2021)
def gen_lmp_smpl(smpl_num, smpl_filename):
    smpls = set([])
    while (len(smpls) < smpl_num):
        lmp_nproc = random.randint(2, (core_num - 1) * (lv_node_num - 1))
        lmp_ppw = random.randint(1, core_num - 1)
        lmp_nthread = random.randint(1, 4)
        lmp_io_step = random.randint(1, 8) * 50
        if (lmp_nproc >= lmp_ppw):
            if (lmp_nproc % lmp_ppw == 0):
                nodes = lmp_nproc // lmp_ppw
            else:
                nodes = lmp_nproc // lmp_ppw + 1
            if (nodes <= lv_node_num - 1):
                smpls.add((lmp_nproc, lmp_ppw, lmp_nthread, lmp_io_step))
    smpls_df = pd.DataFrame(data = list(smpls), columns=('lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step'))
    print smpls_df.head(10)
    df2csv(smpls_df, smpl_filename)
    
gen_lmp_smpl(10000, "conf_lmp_smpls.csv")
'''

In [ ]:
'''
random.seed(2022)
def gen_voro_smpl(smpl_num, smpl_filename):
    smpls = set([])
    while (len(smpls) < smpl_num):
        voro_nproc = random.randint(2, (core_num - 1) * (lv_node_num - 1))
        voro_ppw = random.randint(1, core_num - 1)
        voro_nthread = random.randint(1, 4)
        lmp_io_step = random.randint(1, 8) * 50
        if (voro_nproc >= voro_ppw):
            if (voro_nproc % voro_ppw == 0):
                nodes = voro_nproc // voro_ppw
            else:
                nodes = voro_nproc // voro_ppw + 1
            if (nodes <= lv_node_num - 1):
                smpls.add((voro_nproc, voro_ppw, voro_nthread, lmp_io_step))
    smpls_df = pd.DataFrame(data = list(smpls), columns=('voro_nproc', 'voro_ppw', 'voro_nthread', 'lmp_io_step'))
    print smpls_df.head(10)
    df2csv(smpls_df, smpl_filename)
    
gen_voro_smpl(10000, "conf_voro_smpls.csv")
'''

In [ ]:
'''
random.seed(2019)
def gen_hs_smpl(hs_smpl_num, hs_smpl_filename, ht_smpl_filename, sw_smpl_filename):
    hs_smpls = set([])
    while (len(hs_smpls) < hs_smpl_num):
        ht_x_nproc = random.randint(2, 32)
        ht_y_nproc = random.randint(2, 32)
        ht_ppw = random.randint(1, core_num - 1)
        ht_io_step = random.randint(1, 8) * 4
        ht_io_buf = random.randint(1, 40)
        sw_nproc = random.randint(2, (core_num - 1) * (hs_node_num - 1))
        sw_ppw = random.randint(1, core_num - 1)
        
        ht_nproc = ht_x_nproc * ht_y_nproc
        if (ht_nproc >= ht_ppw and sw_nproc >= sw_ppw):
            if (ht_nproc % ht_ppw == 0 and sw_nproc % sw_ppw == 0):
                nodes = ht_nproc // ht_ppw + sw_nproc // sw_ppw
            elif (ht_nproc % ht_ppw == 0 or sw_nproc % sw_ppw == 0):
                nodes = ht_nproc // ht_ppw + sw_nproc // sw_ppw + 1
            else:
                nodes = ht_nproc // ht_ppw + sw_nproc // sw_ppw + 2
            if (nodes <= hs_node_num):
                hs_smpls.add((ht_x_nproc, ht_y_nproc, ht_ppw, ht_io_step, ht_io_buf, sw_nproc, sw_ppw))

    hs_smpls_df = pd.DataFrame(data = list(hs_smpls), \
                               columns=('ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf', \
                                        'sw_nproc', 'sw_ppw'))
    print hs_smpls_df.head(10)
    df2csv(hs_smpls_df, hs_smpl_filename)
    
    ht_smpls_df = pd.DataFrame(np.c_[hs_smpls_df.values[:, 0:5]], \
                                columns=('ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf'))
    ht_smpls_df = ht_smpls_df.drop_duplicates().reset_index(drop=True)
    print ht_smpls_df.head(3)
    print ht_smpls_df.shape[0]
    df2csv(ht_smpls_df, ht_smpl_filename)
    
    sw_smpls_df = pd.DataFrame(np.c_[hs_smpls_df.values[:, 5:7], hs_smpls_df.values[:, 3]], \
                                 columns=('sw_nproc', 'sw_ppw', 'ht_io_step'))
    sw_smpls_df = sw_smpls_df.drop_duplicates().reset_index(drop=True)
    print sw_smpls_df.head(3)
    print sw_smpls_df.shape[0]
    df2csv(sw_smpls_df, sw_smpl_filename)

gen_hs_smpl(10000, "conf_hs_smpls.csv", "conf_h_smpls.csv", "conf_s_smpls.csv")
'''

In [ ]:
'''
random.seed(2020)
def gen_hs_smpl_in(hs_smpl_num, hs_smpl_filename):
    hs_smpls = set([])
    while (len(hs_smpls) < hs_smpl_num):
        ht_x = 2048 / (2 ** random.randint(0, 5))
        ht_y = 2048 / (2 ** random.randint(0, 5))
        ht_iter = 1024 / (2 ** random.randint(0, 5))
        ht_x_nproc = random.randint(2, 32)
        ht_y_nproc = random.randint(2, 32)
        ht_ppw = random.randint(1, core_num - 1)
        ht_io_step = random.randint(1, 8) * 4
        ht_io_buf = random.randint(1, 40)
        sw_nproc = random.randint(2, (core_num - 1) * (hs_node_num - 1))
        sw_ppw = random.randint(1, core_num - 1)
        
        ht_nproc = ht_x_nproc * ht_y_nproc
        if (ht_nproc >= ht_ppw and sw_nproc >= sw_ppw):
            if (ht_nproc % ht_ppw == 0 and sw_nproc % sw_ppw == 0):
                nodes = ht_nproc // ht_ppw + sw_nproc // sw_ppw
            elif (ht_nproc % ht_ppw == 0 or sw_nproc % sw_ppw == 0):
                nodes = ht_nproc // ht_ppw + sw_nproc // sw_ppw + 1
            else:
                nodes = ht_nproc // ht_ppw + sw_nproc // sw_ppw + 2
            if (nodes <= hs_node_num):
                hs_smpls.add((ht_x, ht_y, ht_iter, ht_x_nproc, ht_y_nproc, ht_ppw, ht_io_step, ht_io_buf, \
                              sw_nproc, sw_ppw))

    hs_smpls_df = pd.DataFrame(data = list(hs_smpls), \
                               columns=('ht_x', 'ht_y', 'ht_iter', \
                                        'ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf', \
                                        'sw_nproc', 'sw_ppw'))
    print hs_smpls_df.head(10)
    df2csv(hs_smpls_df, hs_smpl_filename)

gen_hs_smpl_in(10000, "conf_hs_in_smpls.csv")
'''

In [ ]:
'''
random.seed(2021)
def gen_ht_smpl(smpl_num, smpl_filename):
    smpls = set([])
    while (len(smpls) < smpl_num):
        ht_x_nproc = random.randint(2, 32)
        ht_y_nproc = random.randint(2, 32)
        ht_ppw = random.randint(1, core_num - 1)
        ht_io_step = random.randint(1, 8) * 4
        ht_io_buf = random.randint(1, 40)
        
        ht_nproc = ht_x_nproc * ht_y_nproc
        if (ht_nproc >= ht_ppw):
            if (ht_nproc % ht_ppw == 0):
                nodes = ht_nproc // ht_ppw
            else:
                nodes = ht_nproc // ht_ppw + 1
            if (nodes <= hs_node_num - 1):
                smpls.add((ht_x_nproc, ht_y_nproc, ht_ppw, ht_io_step, ht_io_buf))
    smpls_df = pd.DataFrame(data = list(smpls), \
                            columns=('ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf'))
    print smpls_df.head(10)
    df2csv(smpls_df, smpl_filename)
    
gen_ht_smpl(10000, "conf_ht_smpls.csv")
'''

In [ ]:
'''
random.seed(2022)
def gen_sw_smpl(smpl_num, smpl_filename):
    smpls = set([])
    while (len(smpls) < smpl_num):
        sw_nproc = random.randint(2, (core_num - 1) * (hs_node_num - 1))
        sw_ppw = random.randint(1, core_num - 1)
        ht_io_step = random.randint(1, 8) * 4
        
        if (sw_nproc >= sw_ppw):
            if (sw_nproc % sw_ppw == 0):
                nodes = sw_nproc // sw_ppw
            else:
                nodes = sw_nproc // sw_ppw + 1
            if (nodes <= hs_node_num - 1):
                smpls.add((sw_nproc, sw_ppw, ht_io_step))
    smpls_df = pd.DataFrame(data = list(smpls), columns=('sw_nproc', 'sw_ppw', 'ht_io_step'))
    print smpls_df.head(10)
    df2csv(smpls_df, smpl_filename)
    
gen_sw_smpl(10000, "conf_sw_smpls.csv")
'''

In [ ]:
def lmp_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn):
            val.append([float(s) for s in l.split()])
    return pd.DataFrame(val, columns=colns)

def voro_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn):
            val.append([float(s) for s in l.split()])
    return pd.DataFrame(val, columns=colns)

def lv_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn):
            val.append([float(s) for s in l.split()[:len(colns)]])
    return pd.DataFrame(val, columns=colns)

def lv_in_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn): 
            val.append([float(s) for s in l.split()[:len(colns)]])
    return pd.DataFrame(val, columns=colns)

lmp_conf_colns = ['lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step']
voro_conf_colns = ['voro_nproc', 'voro_ppw', 'voro_nthread', 'lmp_io_step']
lv_conf_colns = ['lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step', 'voro_nproc', 'voro_ppw', 'voro_nthread']
lv_in_conf_colns = ['lmp_l2s', 'lmp_sld', 'lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step', \
                    'voro_nproc', 'voro_ppw', 'voro_nthread']

lmp_df  = lmp_load(glob.glob('../data/lmp_time_list_rand03.dat'), lmp_conf_colns)
voro_df = voro_load(glob.glob('../data/voro_time_list_rand03.dat'), voro_conf_colns)
#l_df = lmp_load(glob.glob('../data/lmp_time_list_cmb_rand02.dat'), lmp_conf_colns)
#v_df = voro_load(glob.glob('../data/voro_time_list_cmb_rand02.dat'), voro_conf_colns)
lv_df = lv_load(glob.glob('../data/lmp_voro_time_list_rand03.dat'), lv_conf_colns)
lv_in_df = lv_in_load(glob.glob('../data/lmp_voro_in_time_list_rand03.dat'), lv_in_conf_colns)

lmp_df = lmp_df.sort_values(lmp_conf_colns).reset_index(drop=True)
voro_df = voro_df.sort_values(voro_conf_colns).reset_index(drop=True)
#l_df = l_df.sort_values(lmp_conf_colns).reset_index(drop=True)
#v_df = v_df.sort_values(voro_conf_colns).reset_index(drop=True)
lv_df = lv_df.sort_values(lv_conf_colns).reset_index(drop=True)
lv_in_df = lv_in_df.sort_values(lv_in_conf_colns).reset_index(drop=True)

#lv_df = lv_df.sort_values(lv_df.columns.tolist()[:-1]).drop_duplicates(subset=lv_conf_colns).reset_index(drop=True)

In [ ]:
def ht_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn):
            val.append([float(s) for s in l.split()])
    return pd.DataFrame(val, columns=colns)

def sw_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn):
            val.append([float(s) for s in l.split()])
    return pd.DataFrame(val, columns=colns)

def hs_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn):
            val.append([float(s) for s in l.split()[:len(colns)]])
    return pd.DataFrame(val, columns=colns)

def hs_in_load(fns, conf_colns, perfn='run_time'):
    if (perfn == ''):
        colns = conf_colns
    else:
        colns = conf_colns + [perfn]
    val = []
    for fn in fns:
        for l in open(fn): 
            val.append([float(s) for s in l.split()[:len(colns)]])
    return pd.DataFrame(val, columns=colns)

ht_conf_colns = ['ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf']
sw_conf_colns = ['sw_nproc', 'sw_ppw', 'ht_io_step']
hs_conf_colns = ['ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf', 'sw_nproc', 'sw_ppw']
hs_in_conf_colns = ['ht_x', 'ht_y', 'ht_iter', 'ht_x_nproc', 'ht_y_nproc', 'ht_ppw', 'ht_io_step', 'ht_io_buf', \
                    'sw_nproc', 'sw_ppw']

ht_df  = ht_load(glob.glob('../data/ht_time_list_rand01.dat'), ht_conf_colns)
sw_df = sw_load(glob.glob('../data/sw_time_list_rand01.dat'), sw_conf_colns)
hs_df = hs_load(glob.glob('../data/ht_sw_time_list_rand01.dat'), hs_conf_colns)
hs_in_df = hs_in_load(glob.glob('../data/ht_sw_in_time_list_rand01.dat'), hs_in_conf_colns)

ht_df = ht_df.sort_values(ht_conf_colns).reset_index(drop=True)
sw_df = sw_df.sort_values(sw_conf_colns).reset_index(drop=True)
hs_df = hs_df.sort_values(hs_conf_colns).reset_index(drop=True)
hs_in_df = hs_in_df.sort_values(hs_in_conf_colns).reset_index(drop=True)


In [ ]:
def df_intersection(df1, df2, cols):
    df1_idx = df1.set_index(cols).index
    df2_idx = df2.set_index(cols).index
    df1_mask = df1_idx.isin(df2_idx)
    df2_mask = df2_idx.isin(df1_idx)
    df1 = df1.loc[df1_mask]
    df2 = df2.loc[df2_mask]
    intsct_df = df1.reset_index(drop=True)
    return intsct_df

def df_sub(df1, df2, cols):
    intsct_df = df_intersection(df1, df2, cols)
    diff_df = pd.concat([df1, intsct_df]).drop_duplicates(keep=False).reset_index(drop=True)
    return diff_df

def df_union(df1, df2):
    union_df = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)
    return union_df

def df_filter(df, colns, vals):
    num = len(colns)
    if (num != len(vals)):
        print "Error: len(colns) = %d, len(vals) = %d" % (num, len(vals))
    for i in range(num):
        mask = (df[colns[i]].values == vals[i])
        df = df.loc[mask]
    rem_colns = [i for i in df.columns.tolist() if i not in colns]
    return df[rem_colns]

def df_ext(df, colns, vals):
    cols = np.asarray([np.asarray(vals) for i in range(df.shape[0])])
    new_df = pd.DataFrame(np.c_[cols, df.values], columns=colns + df.columns.tolist())
    return new_df

In [ ]:
'''
def gen_undo_conf(conf_fns, perf_fns, undo_fn, load_func, conf_colns):
    conf_df = load_func(glob.glob(conf_fns), conf_colns, '')
    perf_df = load_func(glob.glob(perf_fns), conf_colns)
    conf_df_done = df_intersection(conf_df, perf_df, conf_colns)
    conf_df_undo = pd.concat([conf_df, conf_df_done]).drop_duplicates(keep=False).astype(int)
    df2csv(conf_df_undo, undo_fn)
    
gen_undo_conf('conf_lv_smpls.csv', 'lmp_voro_time_list_rand04.dat', 'conf_lv_smpls_undo.csv', lv_load, lv_conf_colns)
gen_undo_conf('conf_hs_smpls2.csv', 'ht_sw_time_list_rand02.dat', 'conf_hs_smpls2_undo.csv', hs_load, hs_conf_colns)

gen_undo_conf('.csv', '.dat', '.csv', lmp_load, lmp_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', voro_load, voro_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', lv_load, lv_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', lv_in_load, lv_in_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', ht_load, ht_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', sw_load, sw_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', hs_load, hs_conf_colns)
gen_undo_conf('.csv', '.dat', '.csv', hs_in_load, hs_in_conf_colns)
'''

In [ ]:
def get_name(df):
    lmp_colns = lmp_conf_colns + ['run_time']
    voro_colns = voro_conf_colns + ['run_time']
    lv_colns = lv_conf_colns + ['run_time']
    lv_in_colns = lv_in_conf_colns + ['run_time']
    
    ht_colns = ht_conf_colns + ['run_time']
    sw_colns = sw_conf_colns + ['run_time']
    hs_colns = hs_conf_colns + ['run_time']
    hs_in_colns = hs_in_conf_colns + ['run_time']

    if (all([x in df.columns.tolist() for x in lv_in_colns])):
        return 'lv_in'
    elif (all([x in df.columns.tolist() for x in lv_colns])):
        return 'lv'
    elif (all([x in df.columns.tolist() for x in lmp_colns])):
        return 'lmp'
    elif (all([x in df.columns.tolist() for x in voro_colns])):
        return 'voro'
    elif (all([x in df.columns.tolist() for x in hs_in_colns])):
        return 'hs_in'
    elif (all([x in df.columns.tolist() for x in hs_colns])):
        return 'hs'
    elif (all([x in df.columns.tolist() for x in ht_colns])):
        return 'ht'
    elif (all([x in df.columns.tolist() for x in sw_colns])):
        return 'sw'
    else:
        return 'unknown'

In [ ]:
def get_runnable_df(df, conf_colns, perf_coln='run_time'):
    arr = np.ones(df.shape[0])
    for i in range(df.shape[0]):
        if (df[perf_coln].values[i] == float('inf')):
            arr[i] = 0.0
    rem_colns = [i for i in df.columns.tolist() if i not in conf_colns]
    vld_df = pd.DataFrame(np.c_[df[conf_colns].values, arr, df[rem_colns].values], \
                          columns=conf_colns + ['runnable'] + rem_colns)
    return vld_df

def get_obj_df(df, colns, weights):
    if (len(colns) != len(weights)):
        print "Error: in get_obj_df(), colns = %d, weights = %d." % (colns, weights)
    weight_sum = np.array(weights).sum()
    if (weight_sum != 1):
        print "Error: in get_obj_df(), weight_sum = %f (!= 1)." % weight_sum
    obj_arr = np.zeros(df.shape[0])
    for i in range(len(colns)):
        mean_val = get_vld_df(df)[colns[i]].values.mean()
        obj_arr += weights[i] * df[colns[i]].values / mean_val
    obj_df = pd.DataFrame(np.c_[df.values, obj_arr], columns=df.columns.tolist() + ['obj'])
    return obj_df

In [ ]:
def get_mach_time(nproc, ppn, runtime):
    nnode = np.negative(np.floor_divide(np.negative(nproc), ppn))
    return np.multiply(nnode, runtime) * core_num / 3600

def invld_mach_time(nproc, ppn, timeout):
    nnode = np.negative(np.floor_divide(np.negative(nproc), ppn))
    return nnode * timeout * core_num / 3600

def app_mach_time(df, timeout=0.0):
    if (df.shape[0] == 0):
        return 0.0
    
    df_name = get_name(df)
    if (df_name != 'lmp' and df_name != 'voro' and df_name != 'ht' and df_name != 'sw'):
        print "Error: in app_mach_time(), df is not lammps, voro, heat-transfer, or stage-write!"
        return -1.0
    
    vld_df = get_vld_df(df)
    if (df_name == 'lmp'):
        vld_nproc = vld_df['lmp_nproc'].values
        vld_ppn = vld_df['lmp_ppw'].values
    elif (df_name == 'voro'):
        vld_nproc = vld_df['voro_nproc'].values
        vld_ppn = vld_df['voro_ppw'].values
    elif (df_name == 'ht'):
        vld_nproc = vld_df['ht_x_nproc'].values * vld_df['ht_y_nproc'].values
        vld_ppn = vld_df['ht_ppw'].values
    else:
        vld_nproc = vld_df['sw_nproc'].values
        vld_ppn = vld_df['sw_ppw'].values
    vld_runtime = vld_df['run_time'].values
    vld_time = get_mach_time(vld_nproc, vld_ppn, vld_runtime).sum()
    
    invld_df = get_invld_df(df)
    if (df_name == 'lmp'):
        invld_nproc = invld_df['lmp_nproc'].values
        invld_ppn = invld_df['lmp_ppw'].values
    elif (df_name == 'voro'):
        invld_nproc = invld_df['voro_nproc'].values
        invld_ppn = invld_df['voro_ppw'].values
    elif (df_name == 'ht'):
        invld_nproc = invld_df['ht_x_nproc'].values * invld_df['ht_y_nproc'].values
        invld_ppn = invld_df['ht_ppw'].values
    else:
        invld_nproc = invld_df['sw_nproc'].values
        invld_ppn = invld_df['sw_ppw'].values
    if (timeout <= 0.0):
        timeout = max(vld_runtime) * 1.1
    invld_time = invld_mach_time(invld_nproc, invld_ppn, timeout).sum()
    return vld_time + invld_time

def sa_mach_time(df, timeout=0.0):
    if (df.shape[0] == 0):
        return 0.0
    
    df_name = get_name(df)
    if (df_name != 'lv' and df_name != 'lv_in' and df_name != 'hs' and df_name != 'hs_in'):
        print "Error: in sa_mach_time(), df is not lv, lv_in, hs, or hs_in!"
        return -1.0
    
    vld_df = get_vld_df(df)
    if (df_name == 'lv' or df_name == 'lv_in'):
        vld_sim_nproc = vld_df['lmp_nproc'].values
        vld_sim_ppn = vld_df['lmp_ppw'].values
        vld_anal_nproc = vld_df['voro_nproc'].values
        vld_anal_ppn = vld_df['voro_ppw'].values
    else:
        vld_sim_nproc = vld_df['ht_x_nproc'].values * vld_df['ht_y_nproc'].values
        vld_sim_ppn = vld_df['ht_ppw'].values
        vld_anal_nproc = vld_df['sw_nproc'].values
        vld_anal_ppn = vld_df['sw_ppw'].values
    vld_runtime = vld_df['run_time'].values
    vld_time = get_mach_time(vld_sim_nproc, vld_sim_ppn, vld_runtime).sum() \
               + get_mach_time(vld_anal_nproc, vld_anal_ppn, vld_runtime).sum()
    
    invld_df = get_invld_df(df)
    if (df_name == 'lv' or df_name == 'lv_in'):
        invld_sim_nproc = invld_df['lmp_nproc'].values
        invld_sim_ppn = invld_df['lmp_ppw'].values
        invld_anal_nproc = invld_df['voro_nproc'].values
        invld_anal_ppn = invld_df['voro_ppw'].values
    else:
        invld_sim_nproc = invld_df['ht_x_nproc'].values * invld_df['ht_y_nproc'].values
        invld_sim_ppn = invld_df['ht_ppw'].values
        invld_anal_nproc = invld_df['sw_nproc'].values
        invld_anal_ppn = invld_df['sw_ppw'].values
    if (timeout <= 0.0):
        timeout = max(vld_runtime) * 1.1
    invld_time = invld_mach_time(invld_sim_nproc, invld_sim_ppn, timeout).sum() \
                 + invld_mach_time(invld_anal_nproc, invld_anal_ppn, timeout).sum()
    return vld_time + invld_time

def get_exec_mach_df(exec_df):
    if ('mach_time' in exec_df.columns.tolist()):
        return exec_df
    
    df_name = get_name(exec_df)
    if (df_name != 'lmp' and df_name != 'voro' and df_name != 'lv' and df_name != 'lv_in' \
        and df_name != 'ht' and df_name != 'sw' and df_name != 'hs' and df_name != 'hs_in'):
        print "Error: unknown dataframe!"
        return exec_df
    
    runtime = exec_df['run_time'].values
    if (df_name == 'lmp' or df_name == 'voro' or df_name == 'ht' or df_name == 'sw'):
        if (df_name == 'lmp'):
            nproc = exec_df['lmp_nproc'].values
            ppn = exec_df['lmp_ppw'].values
        elif (df_name == 'voro'):
            nproc = exec_df['voro_nproc'].values
            ppn = exec_df['voro_ppw'].values
        elif (df_name == 'ht'):
            nproc = exec_df['ht_x_nproc'].values * exec_df['ht_y_nproc'].values
            ppn = exec_df['ht_ppw'].values
        else:
            nproc = exec_df['sw_nproc'].values
            ppn = exec_df['sw_ppw'].values
        mach_time = get_mach_time(nproc, ppn, runtime)
    else:
        if (df_name == 'lv' or df_name == 'lv_in'):
            sim_nproc = exec_df['lmp_nproc'].values
            sim_ppn = exec_df['lmp_ppw'].values
            anal_nproc = exec_df['voro_nproc'].values
            anal_ppn = exec_df['voro_ppw'].values
        else:
            sim_nproc = exec_df['ht_x_nproc'].values * exec_df['ht_y_nproc'].values
            sim_ppn = exec_df['ht_ppw'].values
            anal_nproc = exec_df['sw_nproc'].values
            anal_ppn = exec_df['sw_ppw'].values
        mach_time = get_mach_time(sim_nproc, sim_ppn, runtime) + get_mach_time(anal_nproc, anal_ppn, runtime)
    
    exec_mach_df = pd.DataFrame(np.c_[exec_df.values, mach_time], columns=exec_df.columns.tolist() + ['mach_time'])
    return exec_mach_df

In [ ]:
def get_vld_df(df):
    return df[(df.runnable == 1.0)]

def get_invld_df(df):
    return df[(df.runnable == 0.0)]

# The index of df should be 0, 1, 2, 3, ... in advance by df = df.reset_index(drop=True)
def get_top_idx(df, coln, topn):
    topn = int(topn)
    if (topn <= 0):
        print "Error: in get_top_idx(), topn <= 0!"
    topn = max(1, min(topn, df.shape[0]))
    col_idx = df.columns.tolist().index(coln)
    top_idx = np.argsort(df.values[:, col_idx])[:topn]
    return top_idx

def gen_top_df(df, coln, topn=1):
    top_idx = get_top_idx(df, coln, max(1, topn))
    top_df = pd.DataFrame(np.c_[df.values[top_idx, :]], columns=df.columns.tolist())
    return top_df

def eval_top_match(pred_df, test_df, conf_colns, perf_coln, fn=''):
    if (pred_df.shape[0] != test_df.shape[0]):
        print "Error: pred_df.shape[0] = %d, test_df.shape[0] = %d" % (pred_df.shape[0], test_df.shape[0])
    top_rs_s = set([])
    for topn in range(1, 11, 1):
        pred_df_top = gen_top_df(pred_df, perf_coln, topn)
        test_df_top = gen_top_df(test_df, perf_coln, topn)
        rs = float(df_intersection(pred_df_top, test_df_top, conf_colns).shape[0]) / topn * 100
        topp = float(topn) / test_df.shape[0]
        top_rs_s.add((topp * 100, topn, rs))
    top_rs_df = pd.DataFrame(data=list(top_rs_s), columns=['top_prec', 'top_num', 'recall_score'])
    top_rs_df = top_rs_df.sort_values(top_rs_df.columns.tolist()[0]).reset_index(drop=True)
    #print top_rs_df
    if (fn != ''):
        df2csv(top_rs_df, fn)
    return top_rs_df
'''
def eval_top_match(pred_df, test_df, conf_colns, perf_coln, fn=''):
    if (pred_df.shape[0] != test_df.shape[0]):
        print "Error: pred_df.shape[0] = %d, test_df.shape[0] = %d" % (pred_df.shape[0], test_df.shape[0])
    top_rs_s = set([])
    for topp in (0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01):
        topn = round(max(1, test_df.shape[0] * topp))
        pred_df_top = gen_top_df(pred_df, perf_coln, topn)
        test_df_top = gen_top_df(test_df, perf_coln, topn)
        rs = float(df_intersection(pred_df_top, test_df_top, conf_colns).shape[0]) / topn * 100
        top_rs_s.add((topp * 100, topn, rs))
    top_rs_df = pd.DataFrame(data=list(top_rs_s), columns=['top_prec', 'top_num', 'recall_score'])
    top_rs_df = top_rs_df.sort_values(top_rs_df.columns.tolist()[0]).reset_index(drop=True)
    #print top_rs_df
    if (fn != ''):
        df2csv(top_rs_df, fn)
    return top_rs_df
'''

In [ ]:
lv_full_df = lv_load(glob.glob('../data/lmp_voro_time_list_rand*'), lv_conf_colns)
lv_shrt_df1 = lv_load(glob.glob('../data/sht_lmp_voro_time_list_rand.dat'), lv_conf_colns)
lv_shrt_df2 = lv_load(glob.glob('../data/sht_lmp_voro_time_list_0360_rand.dat'), lv_conf_colns)

lv_full_df = df_intersection(lv_full_df, lv_shrt_df1, lv_conf_colns)
lv_shrt_df1 = df_intersection(lv_shrt_df1, lv_full_df, lv_conf_colns)

lv_full_df = df_intersection(lv_full_df, lv_shrt_df2, lv_conf_colns)
lv_shrt_df2 = df_intersection(lv_shrt_df2, lv_full_df, lv_conf_colns)

#print eval_top_match(lv_shrt_df1, lv_full_df, lv_conf_colns, 'run_time', "../plot/motivation/top_mr_shrt1.csv")
#print eval_top_match(lv_shrt_df2, lv_full_df, lv_conf_colns, 'run_time', "../plot/motivation/top_mr_shrt2.csv")
#print "%3.4f" % sa_mach_time(lv_full_df)
#print "%3.4f (%3.1f%%)" % (sa_mach_time(lv_shrt_df1), 100 * sa_mach_time(lv_shrt_df1) / sa_mach_time(lv_full_df))
#print "%3.4f (%3.1f%%)" % (sa_mach_time(lv_shrt_df2), 100 * sa_mach_time(lv_shrt_df2) / sa_mach_time(lv_full_df))

In [ ]:
obj_perf_colns = ['run_time', 'mach_time']
obj_weights = [0.5, 0.5]

lmp_df = get_obj_df(get_exec_mach_df(get_runnable_df(lmp_df, lmp_conf_colns)), obj_perf_colns, obj_weights)
voro_df = get_obj_df(get_exec_mach_df(get_runnable_df(voro_df, voro_conf_colns)), obj_perf_colns, obj_weights)
lv_df = get_obj_df(get_exec_mach_df(get_runnable_df(lv_df, lv_conf_colns)), obj_perf_colns, obj_weights)
lv_in_df = get_obj_df(get_exec_mach_df(get_runnable_df(lv_in_df, lv_in_conf_colns)), obj_perf_colns, obj_weights)
lv_shrt_df1 = get_obj_df(get_exec_mach_df(get_runnable_df(lv_shrt_df1, lv_conf_colns)), obj_perf_colns, obj_weights)
lv_shrt_df2 = get_obj_df(get_exec_mach_df(get_runnable_df(lv_shrt_df2, lv_conf_colns)), obj_perf_colns, obj_weights)

ht_df = get_obj_df(get_exec_mach_df(get_runnable_df(ht_df, ht_conf_colns)), obj_perf_colns, obj_weights)
sw_df = get_obj_df(get_exec_mach_df(get_runnable_df(sw_df, sw_conf_colns)), obj_perf_colns, obj_weights)
hs_df = get_obj_df(get_exec_mach_df(get_runnable_df(hs_df, hs_conf_colns)), obj_perf_colns, obj_weights)
hs_in_df = get_obj_df(get_exec_mach_df(get_runnable_df(hs_in_df, hs_in_conf_colns)), obj_perf_colns, obj_weights)


In [ ]:
from pandas.util.testing import assert_frame_equal

def is_same_df(df1, df2):
    try:
        assert_frame_equal(df1, df2)
        return True
    except:
        return False

In [ ]:
'''
def get_pdf(df, n_rng):
    min_val = min(df.values[:, -1])
    max_val = np.nextafter(max(df.values[:, -1]), float('inf'))
    step = (max_val - min_val) / n_rng
    bnd_arr = np.array([])
    num_arr = np.array([])
    for i in range(n_rng):
        lower = min_val + step * i
        upper = min_val + step * (i + 1)
        bnd_arr = np.append(bnd_arr, lower)
        num = df[(df.run_time >= lower) & (df.run_time < upper)].shape[0]
        num_arr = np.append(num_arr, num)
    bnd_arr = np.append(bnd_arr, max(df.values[:, -1]))
    num_arr = np.append(num_arr, 0)
    dist_df = pd.DataFrame(np.c_[bnd_arr, num_arr], columns=['bound', 'num'])
    print dist_df
    return dist_df
'''
def get_exec_df(df):
    exec_df = pd.DataFrame(np.c_[df['run_time'].values], columns=['run_time'])
    return exec_df

def get_mach_df(df):
    mach_df = pd.DataFrame(np.c_[df['mach_time'].values], columns=['mach_time'])
    return mach_df

def get_objective_df(df):
    obj_df = pd.DataFrame(np.c_[df['obj'].values], columns=['obj'])
    return obj_df

vld_lv_df = get_vld_df(lv_df)
df2csv(get_exec_df(vld_lv_df), '../plot/distribution/lv_exec_dist.csv')
df2csv(get_mach_df(vld_lv_df), '../plot/distribution/lv_mach_dist.csv')
df2csv(get_objective_df(vld_lv_df), '../plot/distribution/lv_obj_dist.csv')
df2csv(get_mach_df(vld_lv_df[vld_lv_df.run_time <= 60.0]), '../plot/distribution/lv_mach_exec_dist.csv')

vld_lmp_df = get_vld_df(lmp_df)
df2csv(get_exec_df(vld_lmp_df), '../plot/distribution/lmp_exec_dist.csv')
df2csv(get_mach_df(vld_lmp_df), '../plot/distribution/lmp_mach_dist.csv')
df2csv(get_objective_df(vld_lmp_df), '../plot/distribution/lmp_obj_dist.csv')
df2csv(get_mach_df(vld_lmp_df[vld_lmp_df.run_time <= 60.0]), '../plot/distribution/lmp_mach_exec_dist.csv')

vld_voro_df = get_vld_df(voro_df)
df2csv(get_exec_df(vld_voro_df), '../plot/distribution/voro_exec_dist.csv')
df2csv(get_mach_df(vld_voro_df), '../plot/distribution/voro_mach_dist.csv')
df2csv(get_objective_df(vld_voro_df), '../plot/distribution/voro_obj_dist.csv')
df2csv(get_mach_df(vld_voro_df[vld_voro_df.run_time <= 20.0]), '../plot/distribution/voro_mach_exec_dist.csv')

In [ ]:
def plt_err_ts(df, perf_coln):
    perf_coli = df.columns.tolist().index(perf_coln)
    errs = []
    train_rate = (0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5)
    for tr in train_rate:
        train_X, test_X, train_y, test_y = train_test_split(df.values[:, :perf_coli], df.values[:, perf_coli], \
                                                            test_size=1-tr, random_state=2019)
        mdl = xgb.XGBRegressor(max_depth=10, n_estimators=200).fit(train_X, train_y)
        y_p = mdl.predict(test_X)
        abs_rerr = 100. * np.abs(y_p - test_y) / test_y
        errs.append(abs_rerr)
        
    print([(np.percentile(v, 25), np.percentile(v, 50), np.percentile(v, 75), ) for v in errs])
    '''
    ax = new_fig()
    boxprops = dict(linewidth=3, color='black')
    medianprops = dict(linestyle='--', linewidth=3, color='darkgoldenrod')
    capprops = dict(linewidth=2, color='black')
    whiskerprops = dict(linewidth=2, color='black')    
    bplot = plt.boxplot(errs, positions=range(len(train_rate)), widths=0.6, boxprops=boxprops, \
                        medianprops=medianprops, capprops=capprops, whiskerprops=whiskerprops, \
                        patch_artist=True, showfliers=False, notch=True)   

    for patch in bplot['boxes']: patch.set_facecolor('gray')     
    plt.xticks(range(len(train_rate)), ["%d" % (100 *_p) for _p in train_rate])
    plt.xlabel('Training Size (%)', fontsize=20)
    plt.ylabel('Relative Error (%)', fontsize=20)
    plt.ylim((0, 300))
    plt.show()
    plt.close()
    '''

#plt_err_ts(get_vld_df(lv_df), 'run_time')
#plt_err_ts(get_vld_df(lmp_df), 'run_time')
#plt_err_ts(get_vld_df(voro_df), 'run_time')

In [ ]:
'''
lv_df_copy = lv_df.copy()
lv_df_copy = df_intersection(lv_df_copy, l_df, lmp_conf_colns)
lv_df_copy = df_intersection(lv_df_copy, v_df, voro_conf_colns)
lv_df_copy = lv_df_copy.reset_index(drop=True)

l_perf_arr = np.array([])
for i in range(lv_df_copy.shape[0]):
    row_df = df_intersection(l_df, lv_df_copy.head(i + 1).tail(1), lmp_conf_colns)
    lp = row_df.values[0, -1]
    l_perf_arr = np.append(l_perf_arr, lp)

v_perf_arr = np.array([])
for i in range(lv_df_copy.shape[0]):
    row_df = df_intersection(v_df, lv_df_copy.head(i + 1).tail(1), voro_conf_colns)
    vp = row_df.values[0, -1]
    v_perf_arr = np.append(v_perf_arr, vp)

lv_conf = lv_df_copy[lv_conf_colns].values
print "Predict by summing up:"
lv_sum_arr = l_perf_arr + v_perf_arr
lv_sum_df = pd.DataFrame(np.c_[lv_conf, lv_sum_arr], columns=lv_df_copy.columns.tolist())
print eval_top_match(lv_sum_df, lv_df_copy, lv_conf_colns, 'run_time', "../plot/motivation/top_mr_sprt1.csv")

print "Predict based on maximum:"
lv_max_arr = np.concatenate(([l_perf_arr], [v_perf_arr]), axis=0).max(axis=0)
lv_max_df = pd.DataFrame(np.c_[lv_conf, lv_max_arr], columns=lv_df_copy.columns.tolist())
print eval_top_match(lv_max_df, lv_df_copy, lv_conf_colns, 'run_time', "../plot/motivation/top_mr_sprt2.csv")

print "Predict by combining both:"
lv_cmb_arr = 0.5 * lv_sum_arr + 0.5 * lv_max_arr
lv_cmb_df = pd.DataFrame(np.c_[lv_conf, lv_cmb_arr], columns=lv_df_copy.columns.tolist())
print eval_top_match(lv_cmb_df, lv_df_copy, lv_conf_colns, 'run_time', "../plot/motivation/top_mr_sprt3.csv")
'''

In [ ]:
def get_rank(df, coln, val):
    sort_df = gen_top_df(df, coln, df.shape[0]).reset_index(drop=True)
    return sort_df[coln].values.tolist().index(val)

def find_top_rank(top_smpl, df, conf_colns, perf_coln):
    top_cnddt = df_intersection(df, top_smpl, conf_colns)
    top = gen_top_df(top_cnddt, perf_coln)
    print top[perf_coln].values[0]
    print get_rank(df, perf_coln, top[perf_coln].values[0])

def eval_fail(pred_df, test_df, coln='runnable'):
    pred_y = pred_df[coln].values
    test_y = test_df[coln].values
    test_vld_num = np.array(filter(lambda x: x == 1.0, test_y)).sum()
    pred_vld_num = np.array(filter(lambda x: x == 1.0, pred_y)).sum()
    vld2invld_num = np.array(filter(lambda x: x == 1.0, test_y - pred_y)).sum()
    invld2vld_num = np.array(filter(lambda x: x == 1.0, pred_y - test_y)).sum()
    recall = 100. * (test_vld_num - vld2invld_num) / test_vld_num
    precision = 100. * (pred_vld_num - invld2vld_num) / pred_vld_num
    #print "recall = %3.1f%%, precision = %3.1f%%" % (recall, precision)
    return recall, precision
    
def eval_err(pred_df, test_df, perf_coln):
    pred_y = pred_df[perf_coln].values
    test_y = test_df[perf_coln].values
    
    abs_rerr = 100. * np.abs(pred_y - test_y) / test_y
    err_prcntl = np.array([])
    for i in range(0, 101, 10):
        err_prcntl = np.append(err_prcntl, np.percentile(abs_rerr, i))
    err_prcntl_2d = [err_prcntl]
    
    for topp in (0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5):
        top_idx = get_top_idx(test_df, perf_coln, max(1, test_df.shape[0] * topp))
        abs_rerr_top = 100. * np.abs(pred_y[top_idx] - test_y[top_idx]) / test_y[top_idx]
        
        err_prcntl = np.array([])
        for i in range(0, 101, 10):
            err_prcntl = np.append(err_prcntl, np.percentile(abs_rerr_top, i))
        err_prcntl_2d = np.append(err_prcntl_2d, [err_prcntl], axis=0)
    
    err_prcntl_2d = np.append(err_prcntl_2d, [err_prcntl_2d[0, :]], axis=0)
    err_prcntl_2d = np.delete(err_prcntl_2d, (0), axis=0)
    top = 100. * np.array([0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0])
    err_prcntl_2d = np.append(np.transpose([top]), err_prcntl_2d, axis=1)
    err_prcntl_df = pd.DataFrame(np.c_[err_prcntl_2d], columns=['top_perc'] + range(0, 101, 10))
    #print err_prcntl_df
    return err_prcntl_df
    
def train_mdl(train_df, conf_colns, perf_coln):
    train_X = train_df[conf_colns].values
    train_y = train_df[perf_coln].values
    mdl = xgb.XGBRegressor(max_depth=10, n_estimators=200).fit(train_X, train_y)
    return mdl

def train_mdl_chk(train_df, conf_colns, perf_coln):
    mdl_chk = train_mdl(train_df, conf_colns, 'runnable')
    train_df_vld = get_vld_df(train_df)
    mdl = train_mdl(train_df_vld, conf_colns, perf_coln)
    return mdl_chk, mdl

In [ ]:
def xgb_kfold(*dfs_conf_perf):
    err_prcntl_arr = [np.zeros(11)]
    for df_tuple in dfs_conf_perf:
        df = df_tuple[0]
        conf_colns = df_tuple[1]
        perf_coln = df_tuple[2]
        X, y = df[conf_colns].values, df[perf_coln].values
        kf = sklearn.model_selection.KFold(n_splits = 5, random_state=2019, shuffle=True)
        err_prcntl_2d = [np.zeros(11)]
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            mdl = xgb.XGBRegressor(max_depth=10, n_estimators=200).fit(X_train, y_train)
            y_p = mdl.predict(X_test)
            abs_rerr = 100. * np.abs(y_p - y_test) / y_test
        
            err_prcntl = np.array([])
            for i in range(0, 101, 10):
                err_prcntl = np.append(err_prcntl, np.percentile(abs_rerr, i))
            err_prcntl_2d = np.append(err_prcntl_2d, [err_prcntl], axis=0)
            
        err_prcntl_2d = np.delete(err_prcntl_2d, (0), axis=0)
        err_prcntl_arr = np.append(err_prcntl_arr, [np.mean(err_prcntl_2d, axis=0)], axis=0)
        
    err_prcntl_arr = np.delete(err_prcntl_arr, (0), axis=0)
    err_prcntl_df = pd.DataFrame(np.c_[err_prcntl_arr], columns=range(0, 101, 10))
    print err_prcntl_df
    df2csv(err_prcntl_df, "../plot/motivation/err_prcntl.csv")

xgb_kfold((get_vld_df(lv_df).head(449), lv_conf_colns, 'run_time'), \
          (get_vld_df(lv_in_df).head(449), lv_in_conf_colns, 'run_time'))
print sa_mach_time(get_vld_df(lv_df).head(449)) / 449
print sa_mach_time(get_vld_df(lv_in_df).head(449)) / 449

In [ ]:
def sprt_cmb_pred_vld(pred_y1, pred_y2):
    pred_y = np.concatenate(([pred_y1], [pred_y2]), axis=0).min(axis=0)
    return pred_y

def sprt_cmb_pred_val(pred_y1, pred_y2, perf_coln):
    if (perf_coln == 'run_time'):
        pred_y = 0.5 * (pred_y1 + pred_y2) + 0.5 * np.concatenate(([pred_y1], [pred_y2]), axis=0).max(axis=0)
    elif (perf_coln == 'mach_time'):
        pred_y = pred_y1 + pred_y2
    else:
        pred_y = np.concatenate(([pred_y1], [pred_y2]), axis=0).max(axis=0)
    return pred_y

def sprt_pred_chk(mdl1_chk, mdl2_chk, test_df, conf1_colns, conf2_colns, conf_colns):
    test_X1_chk = test_df[conf1_colns].values
    test_X2_chk = test_df[conf2_colns].values
    pred_y1_chk = mdl1_chk.predict(test_X1_chk)
    pred_y2_chk = mdl2_chk.predict(test_X2_chk)
    pred_y_chk = sprt_cmb_pred_vld(pred_y1_chk, pred_y2_chk)
    for i in range(test_df.shape[0]):
        if (pred_y_chk[i] > 0.0005):
            pred_y_chk[i] = 1.0
        else:
            pred_y_chk[i] = 0.0
    pred_df_chk = pd.DataFrame(np.c_[test_df[conf_colns].values, pred_y_chk], columns=conf_colns + ['runnable'])
    return pred_df_chk
    
def sprt_pred_val(pred_df_chk, mdl1, mdl2, test_df, conf1_colns, conf2_colns, conf_colns, perf_coln):
    test_df_vld = get_vld_df(test_df)
    test_X1_vld = test_df_vld[conf1_colns].values
    test_X2_vld = test_df_vld[conf2_colns].values
    pred_y1_vld = mdl1.predict(test_X1_vld)
    pred_y2_vld = mdl2.predict(test_X2_vld)
    pred_y_vld = sprt_cmb_pred_val(pred_y1_vld, pred_y2_vld, perf_coln)
    pred_df_vld = pd.DataFrame(np.c_[test_df_vld[conf_colns].values, pred_y_vld], columns=conf_colns + [perf_coln])
    
    pred_df = pred_df_chk[(pred_df_chk.runnable == 1.0)]
    test_X1 = pred_df[conf1_colns].values
    test_X2 = pred_df[conf2_colns].values
    pred_y1 = mdl1.predict(test_X1)
    pred_y2 = mdl2.predict(test_X2)
    pred_y = sprt_cmb_pred_val(pred_y1, pred_y2, perf_coln)
    pred_df = pd.DataFrame(np.c_[pred_df[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df_invld = pred_df_chk[(pred_df_chk.runnable == 0.0)]
    pred_y = pred_df_invld['runnable'].values + np.ones(pred_df_invld.shape[0]) * float('inf')
    pred_df_invld = pd.DataFrame(np.c_[pred_df_invld[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df = pred_df.append(pred_df_invld)
    return pred_df, pred_df_vld
    
def sprt_pred_top_eval(train_df1, train_df2, test_df, conf1_colns, conf2_colns, conf_colns, perf_coln, \
                       topn=1, eval_flag=1):
    mdl1_chk, mdl1 = train_mdl_chk(train_df1, conf1_colns, perf_coln)
    mdl2_chk, mdl2 = train_mdl_chk(train_df2, conf2_colns, perf_coln)
    
    pred_df_chk = sprt_pred_chk(mdl1_chk, mdl2_chk, test_df, conf1_colns, conf2_colns, conf_colns)
    test_df_vld = get_vld_df(test_df)
    
    if (perf_coln != 'obj'):
        pred_df, pred_df_vld = sprt_pred_val(pred_df_chk, mdl1, mdl2, test_df, \
                                             conf1_colns, conf2_colns, conf_colns, perf_coln)
    else:
        obj_vld_arr = np.zeros(test_df_vld.shape[0])
        obj_arr = np.zeros(test_df.shape[0])
        for i in range(len(obj_perf_colns)):
            pred_df, pred_df_vld = sprt_pred_val(pred_df_chk, mdl1, mdl2, test_df, \
                                                 conf1_colns, conf2_colns, conf_colns, obj_perf_colns[i])
            mean_val = test_df_vld[obj_perf_colns[i]].values.mean()
            obj_vld_arr += obj_weights[i] * pred_df_vld[obj_perf_colns[i]].values / mean_val
            obj_arr += obj_weights[i] * pred_df[obj_perf_colns[i]].values / mean_val
        pred_df_vld = pd.DataFrame(np.c_[test_df_vld[conf_colns].values, obj_vld_arr], columns=conf_colns + ['obj'])
        pred_df = pd.DataFrame(np.c_[test_df[conf_colns].values, obj_arr], columns=conf_colns + ['obj'])
    
    top_smpl = gen_top_df(pred_df, perf_coln, topn)
    if (eval_flag != 0):
        recall, precision = eval_fail(pred_df_chk, test_df)
        err_prcntl_df = eval_err(pred_df_vld, test_df_vld, perf_coln)
        eval_top_match(pred_df_vld, test_df_vld, conf_colns, perf_coln)
        top_rs_df = eval_top_match(pred_df, test_df, conf_colns, perf_coln)
        return top_smpl, err_prcntl_df, top_rs_df
    else:
        return top_smpl

In [ ]:
'''
def sprt_pred_top_eval(train_df1, train_df2, test_df, conf1_colns, conf2_colns, conf_colns, perf_coln, \
                       topn=1, eval_flag=1):
    mdl1_chk, mdl1 = train_mdl_chk(train_df1, conf1_colns, perf_coln)
    mdl2_chk, mdl2 = train_mdl_chk(train_df2, conf2_colns, perf_coln)
    
    test_X1_chk = test_df[conf1_colns].values
    test_X2_chk = test_df[conf2_colns].values
    pred_y1_chk = mdl1_chk.predict(test_X1_chk)
    pred_y2_chk = mdl2_chk.predict(test_X2_chk)
    pred_y_chk = sprt_cmb_pred_vld(pred_y1_chk, pred_y2_chk)
    for i in range(test_df.shape[0]):
        if (pred_y_chk[i] > 0.0005):
            pred_y_chk[i] = 1.0
        else:
            pred_y_chk[i] = 0.0
    pred_df_chk = pd.DataFrame(np.c_[test_df[conf_colns].values, pred_y_chk], columns=conf_colns + ['runnable'])
    
    test_df_vld = get_vld_df(test_df)
    test_X1_vld = test_df_vld[conf1_colns].values
    test_X2_vld = test_df_vld[conf2_colns].values
    pred_y1_vld = mdl1.predict(test_X1_vld)
    pred_y2_vld = mdl2.predict(test_X2_vld)
    pred_y_vld = sprt_cmb_pred_val(pred_y1_vld, pred_y2_vld, perf_coln)
    pred_df_vld = pd.DataFrame(np.c_[test_df_vld[conf_colns].values, pred_y_vld], columns=conf_colns + [perf_coln])
        
    pred_df = pred_df_chk[(pred_df_chk.runnable == 1.0)]
    test_X1 = pred_df[conf1_colns].values
    test_X2 = pred_df[conf2_colns].values
    pred_y1 = mdl1.predict(test_X1)
    pred_y2 = mdl2.predict(test_X2)
    pred_y = sprt_cmb_pred_val(pred_y1, pred_y2, perf_coln)
    pred_df = pd.DataFrame(np.c_[pred_df[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df_invld = pred_df_chk[(pred_df_chk.runnable == 0.0)]
    pred_y = pred_df_invld['runnable'].values + np.ones(pred_df_invld.shape[0]) * float('inf')
    pred_df_invld = pd.DataFrame(np.c_[pred_df_invld[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df = pred_df.append(pred_df_invld)
        
    top_smpl = gen_top_df(pred_df, perf_coln, topn)
    if (eval_flag != 0):
        recall, precision = eval_fail(pred_df_chk, test_df)
        err_prcntl_df = eval_err(pred_df_vld, test_df_vld, perf_coln)
        eval_top_match(pred_df_vld, test_df_vld, conf_colns, perf_coln)
        top_rs_df = eval_top_match(pred_df, test_df, conf_colns, perf_coln)
        return top_smpl, err_prcntl_df, top_rs_df
    else:
        return top_smpl
'''

def whl_pred_top_eval(train_df, test_df, conf_colns, perf_coln, topn=1, eval_flag=1, fn=''):
    mdl_chk, mdl = train_mdl_chk(train_df, conf_colns, perf_coln)
    
    test_X_chk = test_df[conf_colns].values
    pred_y_chk = mdl_chk.predict(test_X_chk)
    for i in range(test_df.shape[0]):
        if (pred_y_chk[i] > 0.0005):
            pred_y_chk[i] = 1.0
        else:
            pred_y_chk[i] = 0.0
    pred_df_chk = pd.DataFrame(np.c_[test_X_chk, pred_y_chk], columns=conf_colns + ['runnable'])
        
    test_df_vld = get_vld_df(test_df)
    test_X_vld = test_df_vld[conf_colns].values
    pred_y_vld = mdl.predict(test_X_vld)
    pred_df_vld = pd.DataFrame(np.c_[test_X_vld, pred_y_vld], columns=conf_colns + [perf_coln])
        
    pred_df = pred_df_chk[(pred_df_chk.runnable == 1.0)]
    test_X = pred_df[conf_colns].values
    pred_y = mdl.predict(test_X)
    pred_df = pd.DataFrame(np.c_[test_X, pred_y], columns=conf_colns + [perf_coln])
    pred_df_invld = pred_df_chk[(pred_df_chk.runnable == 0.0)]
    pred_y = pred_df_invld['runnable'].values + np.ones(pred_df_invld.shape[0]) * float('inf')
    pred_df_invld = pd.DataFrame(np.c_[pred_df_invld[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df = pred_df.append(pred_df_invld)
        
    top_smpl = gen_top_df(pred_df, perf_coln, topn)
    if (eval_flag != 0):
        recall, precision = eval_fail(pred_df_chk, test_df)
        err_prcntl_df = eval_err(pred_df_vld, test_df_vld, perf_coln)
        eval_top_match(pred_df_vld, test_df_vld, conf_colns, perf_coln)
        top_rs_df = eval_top_match(pred_df, test_df, conf_colns, perf_coln)
        perf_df = pd.DataFrame(np.c_[test_df.sort_values(conf_colns)[perf_coln].values, \
                                     pred_df.sort_values(conf_colns)[perf_coln].values], \
                               columns=['real_'+perf_coln, 'pred_'+perf_coln])
        if (fn != ''):
            df2csv(perf_df, fn)
        return top_smpl, err_prcntl_df, top_rs_df
    else:
        return top_smpl
    
def whl_in_pred_top_eval(train_df, test_df, inparams, conf_in_colns, conf_colns, perf_coln, topn=1, eval_flag=1):
    mdl_chk, mdl = train_mdl_chk(train_df, conf_in_colns, perf_coln)
    
    test_num_chk = test_df.shape[0]
    in_X_chk = np.asarray([np.asarray(inparams) for i in range(test_num_chk)])
    test_X_chk = np.concatenate((in_X_chk, test_df[conf_colns].values), axis=1)
    pred_y_chk = mdl_chk.predict(test_X_chk)
    for i in range(test_num_chk):
        if (pred_y_chk[i] > 0.0005):
            pred_y_chk[i] = 1.0
        else:
            pred_y_chk[i] = 0.0
    pred_df_chk = pd.DataFrame(np.c_[test_df[conf_colns].values, pred_y_chk], columns=conf_colns + ['runnable'])
    
    test_df_vld = get_vld_df(test_df)
    test_num_vld = test_df_vld.shape[0]
    in_X_vld = np.asarray([np.asarray(inparams) for i in range(test_num_vld)])
    test_X_vld = np.concatenate((in_X_vld, test_df_vld[conf_colns].values), axis=1)
    pred_y_vld = mdl.predict(test_X_vld)
    pred_df_vld = pd.DataFrame(np.c_[test_df_vld[conf_colns].values, pred_y_vld], columns=conf_colns + [perf_coln])
    
    pred_df = pred_df_chk[(pred_df_chk.runnable == 1.0)]
    test_num = pred_df.shape[0]
    in_X = np.asarray([np.asarray(inparams) for i in range(test_num)])
    test_X = np.concatenate((in_X, pred_df[conf_colns].values), axis=1)
    pred_y = mdl.predict(test_X)
    pred_df = pd.DataFrame(np.c_[pred_df[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df_invld = pred_df_chk[(pred_df_chk.runnable == 0.0)]
    pred_y = pred_df_invld['runnable'].values + np.ones(pred_df_invld.shape[0]) * float('inf')
    pred_df_invld = pd.DataFrame(np.c_[pred_df_invld[conf_colns].values, pred_y], columns=conf_colns + [perf_coln])
    pred_df = pred_df.append(pred_df_invld)
    
    top_smpl = gen_top_df(pred_df, perf_coln, topn)
    if (eval_flag != 0):
        recall, precision = eval_fail(pred_df_chk, test_df)
        err_prcntl_df = eval_err(pred_df_vld, test_df_vld, perf_coln)
        eval_top_match(pred_df_vld, test_df_vld, conf_colns, perf_coln)
        top_rs_df = eval_top_match(pred_df, test_df, conf_colns, perf_coln)
        return top_smpl, err_prcntl_df, top_rs_df
    else:
        return top_smpl

In [ ]:
from scipy.optimize import differential_evolution
import time

def obj_func_lv(x, *params):
    lmp_nproc = round(x[0])
    lmp_ppw = round(x[1])
    lmp_nthread = round(x[2])
    lmp_io_step = round(x[3]) * 50
    voro_nproc = round(x[4])
    voro_ppw = round(x[5])
    voro_nthread = round(x[6])
    x_chk = np.array([lmp_nproc, lmp_ppw, lmp_nthread, lmp_io_step, voro_nproc, voro_ppw, voro_nthread])
    y = float('inf')
    if (lmp_nproc >= lmp_ppw and voro_nproc >= voro_ppw):
        if (lmp_nproc % lmp_ppw == 0 and voro_nproc % voro_ppw == 0):
            nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw
        elif (lmp_nproc % lmp_ppw == 0 or voro_nproc % voro_ppw == 0):
            nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 1
        else:
            nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 2
        if (nodes <= lv_node_num):
            algo = params[0]
            mdls = params[1:]
            if (algo == 'cmtl_ch'):
                mdl1_chk, mdl1, mdl2_chk, mdl2 = mdls[0], mdls[1], mdls[2], mdls[3]
                mdl_chk, mdl = mdls[4], mdls[5]
                y1, y2 = float('inf'), float('inf')
                x1 = x_chk[:4]
                y1_chk = mdl1_chk.predict([x1])[0]
                if (y1_chk >= 0.0005):
                    y1 = mdl1.predict([x1])[0]
                x2 = np.hstack((x_chk[4:7], [x_chk[3]]))
                y2_chk = mdl2_chk.predict([x2])[0]
                if (y2_chk >= 0.0005):
                    y2 = mdl2.predict([x2])[0]
                x_p = np.concatenate((x_chk, np.array([y1, y2])), axis=0)
                y_chk = mdl_chk.predict([x_p])[0]
                if (y_chk >= 0.0005):
                    y = mdl.predict([x_p])[0]
            elif (algo == 'cmtl_wh'):
                in_mdl_chk, in_mdl = mdls[0], mdls[1]
                mdl_chk, mdl = mdls[2], mdls[3]
                y_in = float('inf')
                x_in = np.concatenate((np.array([16000, 20000]), x_chk), axis=0)
                y_in_chk = in_mdl_chk.predict([x_in])[0]
                if (y_in_chk >= 0.0005):
                    y_in = in_mdl.predict([x_in])[0]
                x_p = np.concatenate((x_chk, np.array([y_in])), axis=0)
                y_chk = mdl_chk.predict([x_p])[0]
                if (y_chk >= 0.0005):
                    y = mdl.predict([x_p])[0]
            elif (algo == 'sprt'):
                mdl1_chk, mdl2_chk, mdl1, mdl2 = mdls[0], mdls[1], mdls[2], mdls[3]
                x1 = x_chk[:4]
                x2 = np.hstack((x_chk[4:7], [x_chk[3]]))
                y1_chk = mdl1_chk.predict([x1])[0]
                y2_chk = mdl2_chk.predict([x2])[0]
                y_chk = min(y1_chk, y2_chk)
                if (y_chk >= 0.0005):
                    y1 = mdl1.predict([x1])[0]
                    y2 = mdl2.predict([x2])[0]
                    y = 0.5 * (y1 + y2) + 0.5 * max(y1, y2)
            else:
                mdl_chk = mdls[0]
                mdl = mdls[1]
                y_chk = mdl_chk.predict([x_chk])[0]
                if (y_chk >= 0.0005):
                    y = mdl.predict([x_chk])[0]
    return y

def pred_top_lv(algo, mdls):
    bounds = [(2.0 - 0.1, (core_num - 1.0) * (lv_node_num - 1.0) + 0.1), \
              (1.0 - 0.1, core_num - 1 + 0.1), \
              (1.0 - 0.1, 4.0 + 0.1), \
              (1.0 - 0.1, 8.0 + 0.1), \
              (2.0 - 0.1, (core_num - 1.0) * (lv_node_num - 1.0) + 0.1), \
              (1.0 - 0.1, core_num - 1 + 0.1), \
              (1.0 - 0.1, 4.0 + 0.1)]
    params = (algo, ) + mdls
    result = differential_evolution(obj_func_lv, bounds, args=params)
    x = np.rint(result.x)
    if (algo == 'cmtl_ch'):
        y = obj_func_lv(x, algo, mdls[0], mdls[1], mdls[2], mdls[3], mdls[4], mdls[5])
    elif (algo == 'cmtl_wh' or algo == 'sprt'):
        y = obj_func_lv(x, algo, mdls[0], mdls[1], mdls[2], mdls[3])
    else:
        y = obj_func_lv(x, algo, mdls[0], mdls[1])
    x[3] *= 50
    smpl_arr = np.hstack((x, [y]))
    return smpl_arr

def get_pred_top_smpl(algo, mdls, conf_colns, perfn, topn=10):
    startT = time.time()
    slctR = 0.5
    cnddtn = int(float(topn) / slctR)
    for i in range(cnddtn):
        smpl_arr = pred_top_lv(algo, mdls)
        if (i == 0):
            smpls_arr = smpl_arr
        else:
            smpls_arr = np.vstack((smpls_arr, smpl_arr))
    smpl_df = pd.DataFrame(np.c_[smpls_arr], columns=conf_colns + [perfn])
    top_smpl_df = smpl_df.drop_duplicates().sort_values([perfn]).reset_index(drop=True).head(topn)
    runtime = time.time() - startT
    print "Execution time is %0.4f s." % runtime
    return top_smpl_df

In [ ]:
def pred_perf_chk(X_arr, mdl_chk, mdl):
    pred_y_chk = mdl_chk.predict(X_arr)
    pred_y = mdl.predict(X_arr)
    for i in range(len(pred_y_chk)):
        if (pred_y_chk[i] < 0.5):
            pred_y[i] = float('inf')
    return pred_y

def add_layer_sprt_pred(df, conf1_colns, conf2_colns, conf_colns, perf_coln, mdl1_chk, mdl1, mdl2_chk, mdl2):        
    X1 = df[conf1_colns].values
    pred_y1 = pred_perf_chk(X1, mdl1_chk, mdl1)

    X2 = df[conf2_colns].values
    pred_y2 = pred_perf_chk(X2, mdl2_chk, mdl2)
            
    pred_df = pd.DataFrame(np.c_[df[conf_colns].values, pred_y1, pred_y2, df['runnable'].values, df[perf_coln].values], \
                           columns=conf_colns + [perf_coln+'1', perf_coln+'2', 'runnable', perf_coln])
    return pred_df

def cmbn_sprt_top_eval(train_df1, train_df2, train_df, test_df, conf1_colns, conf2_colns, conf_colns, perf_coln, \
                       topn=1, eval_flag=1):
    mdl1_chk, mdl1 = train_mdl_chk(train_df1, conf1_colns, perf_coln)
    mdl2_chk, mdl2 = train_mdl_chk(train_df2, conf2_colns, perf_coln)
    train_intrmdt_df = add_layer_sprt_pred(train_df, conf1_colns, conf2_colns, conf_colns, perf_coln, \
                                           mdl1_chk, mdl1, mdl2_chk, mdl2)
    mdl_chk, mdl = train_mdl_chk(train_intrmdt_df, conf_colns + [perf_coln+'1', perf_coln+'2'], perf_coln)
    test_intrmdt_df = add_layer_sprt_pred(test_df, conf1_colns, conf2_colns, conf_colns, perf_coln, \
                                          mdl1_chk, mdl1, mdl2_chk, mdl2)
    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_intrmdt_df, test_intrmdt_df, \
                                                                conf_colns + [perf_coln+'1', perf_coln+'2'], perf_coln, \
                                                                topn, eval_flag)
    
    top_smpl = whl_pred_top_eval(train_intrmdt_df, get_vld_df(test_intrmdt_df), \
                                 conf_colns + [perf_coln+'1', perf_coln+'2'], perf_coln, 1, 0)
    top_perf = df_intersection(test_intrmdt_df, top_smpl, conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    top_smpl_df = get_pred_top_smpl('cmtl_ch', (mdl1_chk, mdl1, mdl2_chk, mdl2, mdl_chk, mdl, ), \
                                    conf_colns, perf_coln, topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[conf_colns].astype(int)
    '''
    return pred_top_smpl, err_prcntl_df, top_rs_df, top_perf

def cmbn_s_a(sim_df, anal_df, sa_df, sim_conf_colns, anal_conf_colns, sa_conf_colns, perf_coln, smpl_num):
    pred_top_smpl, err_prcntl_df, top_rs_df, top_perf = cmbn_sprt_top_eval(sim_df, anal_df, sa_df.head(smpl_num), \
                                                                           sa_df, sim_conf_colns, anal_conf_colns, \
                                                                           sa_conf_colns, perf_coln, \
                                                                           sa_df.shape[0] * 0.005)
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    machTime = sa_mach_time(sa_df.head(smpl_num), sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, top_perf

In [ ]:
def add_layer_shrt_pred(df, inparams, conf_colns, perf_coln, mdl_chk, mdl):
    in_X = np.asarray([np.asarray(inparams) for i in range(df.shape[0])])
    X = np.concatenate((in_X, df[conf_colns].values), axis=1)
    pred_y = pred_perf_chk(X, mdl_chk, mdl)
    pred_df = pd.DataFrame(np.c_[df[conf_colns].values, pred_y, df['runnable'].values, df[perf_coln].values], \
                           columns=conf_colns + ['pre_'+perf_coln, 'runnable', perf_coln])
    return pred_df

def cmbn_shrt_top_eval(train_in_df, train_df, test_df, inparams, conf_in_colns, conf_colns, perf_coln, \
                       topn=1, eval_flag=1):
    in_mdl_chk, in_mdl = train_mdl_chk(train_in_df, conf_in_colns, perf_coln)
    train_intrmdt_df = add_layer_shrt_pred(train_df, inparams, conf_colns, perf_coln, in_mdl_chk, in_mdl)
    mdl_chk, mdl = train_mdl_chk(train_intrmdt_df, conf_colns + ['pre_'+perf_coln], perf_coln)
    test_intrmdt_df = add_layer_shrt_pred(test_df, inparams, conf_colns, perf_coln, in_mdl_chk, in_mdl)
    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_intrmdt_df, test_intrmdt_df, \
                                                                conf_colns + ['pre_'+perf_coln], perf_coln, \
                                                                topn, eval_flag)
    
    top_smpl = whl_pred_top_eval(train_intrmdt_df, get_vld_df(test_intrmdt_df), \
                                 conf_colns + ['pre_'+perf_coln], perf_coln, 1, 0)
    top_perf = df_intersection(test_intrmdt_df, top_smpl, conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    top_smpl_df = get_pred_top_smpl('cmtl_wh', (in_mdl_chk, in_mdl, mdl_chk, mdl, ), conf_colns, perf_coln, topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[conf_colns].astype(int)
    '''
    return pred_top_smpl, err_prcntl_df, top_rs_df, top_perf

def cmbn_sap(sa_in_df, sa_df, inparams, sa_in_conf_colns, sa_conf_colns, perf_coln, smpl_num):
    pred_top_smpl, err_prcntl_df, top_rs_df, top_perf = cmbn_shrt_top_eval(sa_in_df, sa_df.head(smpl_num), sa_df, \
                                                                           inparams, sa_in_conf_colns, sa_conf_colns, \
                                                                           perf_coln, sa_df.shape[0] * 0.005)
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)   
    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    machTime = sa_mach_time(sa_df.head(smpl_num), sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, top_perf

In [ ]:
def drct_sa(sa_df, sa_conf_colns, perf_coln, smpl_num):
    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(sa_df.head(smpl_num), sa_df, sa_conf_colns, perf_coln, \
                                                                sa_df.shape[0] * 0.005)
    #pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(sa_df.head(smpl_num), sa_df, sa_conf_colns, perf_coln, \
    #                                                            sa_df.shape[0] * 0.005, 1, \
    #                                                            "../plot/run_time/time_lv.csv")
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    top_smpl = whl_pred_top_eval(sa_df.head(smpl_num), get_vld_df(sa_df), sa_conf_colns, perf_coln, 1, 0)
    top_perf = df_intersection(sa_df, top_smpl, sa_conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    train_df = sa_df.head(smpl_num)
    mdl_chk, mdl = train_mdl_chk(train_df, sa_conf_colns, perf_coln)
    top_smpl_df = get_pred_top_smpl('baseline', (mdl_chk, mdl, ), sa_conf_colns, perf_coln, topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[sa_conf_colns].astype(int)
    #df2csv(top_conf_df, "prdct_lv.csv")
    '''
    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    machTime = sa_mach_time(sa_df.head(smpl_num), sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, top_perf

In [ ]:
def sa_sa(sa_df, sa_conf_colns, perf_coln, smpl_num, rand_prec, iter_num):
    rand_sn = int(smpl_num * rand_prec)
    snpi = int((smpl_num - rand_sn) / iter_num)
    train_df = sa_df.head(rand_sn)
    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, smpl_num)
    iter_err = np.array([err_prcntl_df[50].loc[range(4)].values])
    iter_rs = np.array([top_rs_df['recall_score'].values])

    for iter_idx in range(iter_num):
        curr_sn = smpl_num - snpi * (iter_num - 1 - iter_idx)
        
        pred_top_smpl = pred_top_smpl.sort_values([perf_coln]).reset_index(drop=True)
        slct_train_df = df_intersection(sa_df, pred_top_smpl.head(snpi), sa_conf_colns)
        train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        
        last = snpi
        while (train_df.shape[0] < curr_sn):
            last = last + 1
            slct_train_df = df_intersection(sa_df, pred_top_smpl.head(last).tail(1), sa_conf_colns)
            train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, smpl_num)
        iter_err = np.append(iter_err, [err_prcntl_df[50].loc[range(4)].values], axis=0)
        iter_rs = np.append(iter_rs, [top_rs_df['recall_score'].values], axis=0)

        #pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, \
        #                                                            sa_df.shape[0] * 0.005, 1, \
        #                                                            "../plot/run_time/time_lv_lv.csv")
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    iter_err_df = pd.DataFrame(np.transpose(iter_err), columns=range(iter_num + 1))
    iter_rs_df = pd.DataFrame(np.transpose(iter_rs), columns=range(iter_num + 1))

    top_smpl = whl_pred_top_eval(train_df, get_vld_df(sa_df), sa_conf_colns, perf_coln, 1, 0)
    top_perf = df_intersection(sa_df, top_smpl, sa_conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    mdl_chk, mdl = train_mdl_chk(train_df, sa_conf_colns, perf_coln)
    top_smpl_df = get_pred_top_smpl((mdl_chk, mdl, ), topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[sa_conf_colns].astype(int)
    df2csv(top_conf_df, "prdct_lv_lv.csv")
    '''
    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    machTime = sa_mach_time(train_df, sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, iter_err_df, iter_rs_df, top_perf

In [ ]:
def sap_sa(sa_in_df, sa_df, inparams, sa_in_conf_colns, sa_conf_colns, perf_coln, smpl_num, \
           init_prec, rand_prec, iter_num):
    init_sn = int(smpl_num * init_prec)
    rand_sn = int(smpl_num * rand_prec)
    snpi = int((smpl_num - init_sn - rand_prec) / iter_num)

    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    sa_in_timeout = max(get_vld_df(sa_in_df)['run_time'].values) * 1.1
    avg_sa_mach_time = sa_mach_time(sa_df.head(rand_sn), sa_timeout) / rand_sn
    avg_sa_in_mach_time = sa_mach_time(sa_in_df.head(init_sn), sa_in_timeout) / init_sn
    factor = int(max(1, avg_sa_mach_time / avg_sa_in_mach_time))
    
    sa_in_smpl_df = sa_in_df.head(init_sn * factor)
    rem_colns = [i for i in sa_in_conf_colns if i not in sa_conf_colns]
    sa_smpl_df = df_filter(sa_in_smpl_df, rem_colns, inparams)
    
    train_df = sa_df.head(rand_sn)
    sa_in_smpl_df = pd.concat([df_ext(train_df, rem_colns, inparams), \
                               sa_in_smpl_df]).drop_duplicates().reset_index(drop=True)
    
    pred_top_smpl, err_prcntl_df, top_rs_df = whl_in_pred_top_eval(sa_in_smpl_df, sa_df, inparams, \
                                                                   sa_in_conf_colns, sa_conf_colns, perf_coln, smpl_num)
    iter_err = np.array([err_prcntl_df[50].loc[range(4)].values])
    iter_rs = np.array([top_rs_df['recall_score'].values])
    train_df = pd.concat([train_df, sa_smpl_df]).drop_duplicates().reset_index(drop=True)

    for iter_idx in range(iter_num):
        curr_sn = smpl_num - init_sn - snpi * (iter_num - 1 - iter_idx)
        
        pred_top_smpl = pred_top_smpl.sort_values([perf_coln]).reset_index(drop=True)
        slct_train_df = df_intersection(sa_df, pred_top_smpl.head(snpi), sa_conf_colns)
        train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        
        last = snpi
        while (train_df.shape[0] < curr_sn + sa_smpl_df.shape[0]):
            last = last + 1
            slct_train_df = df_intersection(sa_df, pred_top_smpl.head(last).tail(1), sa_conf_colns)
            train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, smpl_num)
        iter_err = np.append(iter_err, [err_prcntl_df[50].loc[range(4)].values], axis=0)
        iter_rs = np.append(iter_rs, [top_rs_df['recall_score'].values], axis=0)
        
        #pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, \
        #                                                            sa_df.shape[0] * 0.005, 1, \
        #                                                            "../plot/run_time/time_lvp_lv.csv")
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    iter_err_df = pd.DataFrame(np.transpose(iter_err), columns=range(iter_num + 1))
    iter_rs_df = pd.DataFrame(np.transpose(iter_rs), columns=range(iter_num + 1))

    top_smpl = whl_pred_top_eval(train_df, get_vld_df(sa_df), sa_conf_colns, perf_coln, 1, 0)
    top_perf = df_intersection(sa_df, top_smpl, sa_conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    mdl_chk, mdl = train_mdl_chk(train_df, sa_conf_colns, perf_coln)
    top_smpl_df = get_pred_top_smpl((mdl_chk, mdl, ), topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[sa_conf_colns].astype(int)
    df2csv(top_conf_df, "prdct_lvp_lv.csv")
    '''
    machTime = sa_mach_time(sa_in_df.head(init_sn * factor), sa_in_timeout) \
               + sa_mach_time(train_df, sa_timeout) \
               - sa_mach_time(df_intersection(train_df, sa_in_smpl_df, sa_conf_colns), sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, iter_err_df, iter_rs_df, top_perf

In [ ]:
def s_a_sa(sim_df, anal_df, sa_df, sim_conf_colns, anal_conf_colns, sa_conf_colns, perf_coln, smpl_num, \
           history, init_prec, rand_prec, iter_num):
    sim_timeout = max(get_vld_df(sim_df)['run_time'].values) * 1.1
    anal_timeout = max(get_vld_df(anal_df)['run_time'].values) * 1.1
    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    
    rand_sn = int(smpl_num * rand_prec)
    if (history):
        init_sn = 0
        snpi = int((smpl_num - rand_prec) / iter_num)
        pred_top_smpl, err_prcntl_df, top_rs_df = sprt_pred_top_eval(sim_df, anal_df, sa_df, \
                                                                     sim_conf_colns, anal_conf_colns, sa_conf_colns, \
                                                                     perf_coln, smpl_num)
    else:
        init_sn = int(smpl_num * init_prec)
        snpi = int((smpl_num - init_sn - rand_prec) / iter_num)
                                           
        avg_sa_mach_time = sa_mach_time(sa_df.head(rand_sn), sa_timeout) / rand_sn
        avg_s_a_mach_time = (app_mach_time(sim_df.head(init_sn), sim_timeout) \
                             + app_mach_time(anal_df.head(init_sn), anal_timeout)) / init_sn
        factor = int(max(1, avg_sa_mach_time / avg_s_a_mach_time))
                                           
        pred_top_smpl, err_prcntl_df, top_rs_df = sprt_pred_top_eval(sim_df.head(init_sn * factor), \
                                                                     anal_df.head(init_sn * factor), sa_df, \
                                                                     sim_conf_colns, anal_conf_colns, sa_conf_colns, \
                                                                     perf_coln, smpl_num)
        
    iter_err = np.array([err_prcntl_df[50].loc[range(4)].values])
    iter_rs = np.array([top_rs_df['recall_score'].values])
    train_df = sa_df.head(rand_sn)

    for iter_idx in range(iter_num):
        curr_sn = smpl_num - init_sn - snpi * (iter_num - 1 - iter_idx)
        
        pred_top_smpl = pred_top_smpl.sort_values([perf_coln]).reset_index(drop=True)
        slct_train_df = df_intersection(sa_df, pred_top_smpl.head(snpi), sa_conf_colns)
        train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)

        last = snpi
        while (train_df.shape[0] < curr_sn):
            last = last + 1
            slct_train_df = df_intersection(sa_df, pred_top_smpl.head(last).tail(1), sa_conf_colns)
            train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, smpl_num)
        iter_err = np.append(iter_err, [err_prcntl_df[50].loc[range(4)].values], axis=0)
        iter_rs = np.append(iter_rs, [top_rs_df['recall_score'].values], axis=0)
        
        #if (history):
        #    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, \
        #                                                                sa_df.shape[0] * 0.005, 1, \
        #                                                                "../plot/run_time/time_h_l_v_lv.csv")
        #else:
        #    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, \
        #                                                                sa_df.shape[0] * 0.005, 1, \
        #                                                                "../plot/run_time/time_l_v_lv.csv")
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    iter_err_df = pd.DataFrame(np.transpose(iter_err), columns=range(iter_num + 1))
    iter_rs_df = pd.DataFrame(np.transpose(iter_rs), columns=range(iter_num + 1))

    top_smpl = whl_pred_top_eval(train_df, get_vld_df(sa_df), sa_conf_colns, perf_coln, 1, 0)
    top_perf = df_intersection(sa_df, top_smpl, sa_conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    mdl_chk, mdl = train_mdl_chk(train_df, sa_conf_colns, perf_coln)
    top_smpl_df = get_pred_top_smpl((mdl_chk, mdl, ), topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[sa_conf_colns].astype(int)
    if (history):
        df2csv(top_conf_df, "prdct_h_l_v_lv.csv")
    else:
        df2csv(top_conf_df, "prdct_l_v_lv.csv")
    '''
    if (history):
        machTime = sa_mach_time(train_df, sa_timeout)
    else:
        machTime = app_mach_time(sim_df.head(init_sn * factor), sim_timeout) \
                   + app_mach_time(anal_df.head(init_sn * factor), anal_timeout) \
                   + sa_mach_time(train_df, sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, iter_err_df, iter_rs_df, top_perf

In [ ]:
def s_a(sim_df, anal_df, sa_df, sim_conf_colns, anal_conf_colns, sa_conf_colns, perf_coln, smpl_num, history):
    if (history):
        pred_top_smpl, err_prcntl_df, top_rs_df = sprt_pred_top_eval(sim_df, anal_df, sa_df, \
                                                                     sim_conf_colns, anal_conf_colns, \
                                                                     sa_conf_colns, perf_coln, \
                                                                     sa_df.shape[0] * 0.005)
        top_smpl = sprt_pred_top_eval(sim_df, anal_df, sa_df, sim_conf_colns, anal_conf_colns, sa_conf_colns, \
                                      perf_coln, 1, 0)
    else:
        pred_top_smpl, err_prcntl_df, top_rs_df = sprt_pred_top_eval(sim_df.head(smpl_num), anal_df.head(smpl_num), \
                                                                     sa_df, sim_conf_colns, anal_conf_colns, \
                                                                     sa_conf_colns, perf_coln, \
                                                                     sa_df.shape[0] * 0.005)
        top_smpl = sprt_pred_top_eval(sim_df.head(smpl_num), anal_df.head(smpl_num), sa_df, \
                                      sim_conf_colns, anal_conf_colns, sa_conf_colns, perf_coln, 1, 0)
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    top_perf = df_intersection(sa_df, top_smpl, sa_conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    if (history):
        train_df1 = sim_df
        train_df2 = anal_df
    else:
        train_df1 = sim_df.head(smpl_num)
        train_df2 = anal_df.head(smpl_num)
    mdl1_chk, mdl1 = train_mdl_chk(train_df1, sim_conf_colns, perf_coln)
    mdl2_chk, mdl2 = train_mdl_chk(train_df2, anal_conf_colns, perf_coln)
    top_smpl_df = get_pred_top_smpl((mdl1_chk, mdl2_chk, mdl1, mdl2), topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[sa_conf_colns].astype(int)
    df2csv(top_conf_df, "prdct_lmp_voro.csv")
    '''
    if (history):
        machTime = 0
    else:
        sim_timeout = max(get_vld_df(sim_df)['run_time'].values) * 1.1
        anal_timeout = max(get_vld_df(anal_df)['run_time'].values) * 1.1
        machTime = app_mach_time(sim_df.head(smpl_num), sim_timeout) \
                   + app_mach_time(anal_df.head(smpl_num), anal_timeout)
    return top_rs_df, err_prcntl_df, machTime, top_perf

In [ ]:
def sam_sa(sa_shrt_df, sa_df, sa_conf_colns, perf_coln, smpl_num, init_prec, rand_prec, iter_num):
    sa_timeout = max(get_vld_df(sa_df)['run_time'].values) * 1.1
    sa_shrt_timeout = max(get_vld_df(sa_shrt_df)['run_time'].values) * 1.1
    
    init_sn = int(smpl_num * init_prec)
    rand_sn = int(smpl_num * rand_prec)
    snpi = int((smpl_num - init_sn - rand_prec) / iter_num)
    
    avg_sa_mach_time = sa_mach_time(sa_df.head(rand_sn), sa_timeout) / rand_sn
    avg_sa_shrt_mach_time = sa_mach_time(sa_shrt_df.head(init_sn), sa_shrt_timeout) / init_sn
    factor = int(max(1, avg_sa_mach_time / avg_sa_shrt_mach_time))
    
    pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(sa_shrt_df.head(init_sn * factor), sa_df, \
                                                                sa_conf_colns, perf_coln, smpl_num)
    iter_err = np.array([err_prcntl_df[50].loc[range(4)].values])
    iter_rs = np.array([top_rs_df['recall_score'].values])
    train_df = sa_df.head(rand_sn)
    
    for iter_idx in range(iter_num):
        curr_sn = smpl_num - init_sn - snpi * (iter_num - 1 - iter_idx)
        
        pred_top_smpl = pred_top_smpl.sort_values([perf_coln]).reset_index(drop=True)
        slct_train_df = df_intersection(sa_df, pred_top_smpl.head(snpi), sa_conf_colns)
        train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        
        last = snpi
        while (train_df.shape[0] < curr_sn):
            last = last + 1
            slct_train_df = df_intersection(sa_df, pred_top_smpl.head(last).tail(1), sa_conf_colns)
            train_df = pd.concat([train_df, slct_train_df]).drop_duplicates().reset_index(drop=True)
        pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, smpl_num)
        iter_err = np.append(iter_err, [err_prcntl_df[50].loc[range(4)].values], axis=0)
        iter_rs = np.append(iter_rs, [top_rs_df['recall_score'].values], axis=0)
        
        #pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(train_df, sa_df, sa_conf_colns, perf_coln, \
        #                                                            sa_df.shape[0] * 0.005, 1, \
        #                                                            "../plot/run_time/time_lvm_lv.csv")
    #find_top_rank(pred_top_smpl, sa_df, sa_conf_colns, perf_coln)
    iter_err_df = pd.DataFrame(np.transpose(iter_err), columns=range(iter_num + 1))
    iter_rs_df = pd.DataFrame(np.transpose(iter_rs), columns=range(iter_num + 1))

    top_smpl = whl_pred_top_eval(train_df, get_vld_df(sa_df), sa_conf_colns, perf_coln, 1, 0)
    top_perf = df_intersection(sa_df, top_smpl, sa_conf_colns)[perf_coln].values[0]
    '''
    topn = 10
    mdl_chk, mdl = train_mdl_chk(train_df, sa_conf_colns, perf_coln)
    top_smpl_df = get_pred_top_smpl((mdl_chk, mdl, ), topn)
    print top_smpl_df
    top_conf_df = top_smpl_df[sa_conf_colns].astype(int)
    df2csv(top_conf_df, "prdct_lvm_lv.csv")
    '''
    machTime = sa_mach_time(sa_shrt_df.head(init_sn * factor), sa_shrt_timeout) + sa_mach_time(train_df, sa_timeout)
    return top_rs_df, err_prcntl_df, machTime, iter_err_df, iter_rs_df, top_perf

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/lmp-voro/iteration/' + perf_coln + '/'

in_start = 1
in_end = 7
in_step = 1

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    lmp_df = lmp_df.sort_values(lmp_conf_colns).reset_index(drop=True)
    voro_df = voro_df.sort_values(voro_conf_colns).reset_index(drop=True)
    lv_df = lv_df.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_in_df = lv_in_df.sort_values(lv_in_conf_colns).reset_index(drop=True)

    lmp_df = lmp_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    voro_df = voro_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_df = lv_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_in_df = lv_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # lv + lv => lv
    rand_prec = 0.3
    iter_num = 4
    for iter_num in range(in_start, in_end, in_step):
        res = sa_sa(lv_df, lv_conf_colns, perf_coln, smpl_num, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        rows = top_rs_df[['top_prec', 'top_num']].values
        if (iter_num == in_start):
            lv_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            lv_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            lv_lv_err_prcntl = np.append(lv_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            lv_lv_top_rs = np.append(lv_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        lv_lv_err_prcntl_all = np.array([lv_lv_err_prcntl])
        lv_lv_top_rs_all = np.array([lv_lv_top_rs])
        lv_lv_iter_err_all = np.array([iter_err_df.values])
        lv_lv_iter_rs_all = np.array([iter_rs_df.values])
    else:
        lv_lv_err_prcntl_all = np.append(lv_lv_err_prcntl_all, [lv_lv_err_prcntl], axis=0)
        lv_lv_top_rs_all = np.append(lv_lv_top_rs_all, [lv_lv_top_rs], axis=0)
        lv_lv_iter_err_all = np.append(lv_lv_iter_err_all, [iter_err_df.values], axis=0)
        lv_lv_iter_rs_all = np.append(lv_lv_iter_rs_all, [iter_rs_df.values], axis=0)
    
    # lv+ + lv => lv
    init_prec = 0.1
    rand_prec = 0.5
    iter_num = 2
    for iter_num in range(in_start, in_end, in_step):
        res = sap_sa(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        if (iter_num == in_start):
            lvp_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            lvp_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            lvp_lv_err_prcntl = np.append(lvp_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            lvp_lv_top_rs = np.append(lvp_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        lvp_lv_err_prcntl_all = np.array([lvp_lv_err_prcntl])
        lvp_lv_top_rs_all = np.array([lvp_lv_top_rs])
        lvp_lv_iter_err_all = np.array([iter_err_df.values])
        lvp_lv_iter_rs_all = np.array([iter_rs_df.values])
    else:
        lvp_lv_err_prcntl_all = np.append(lvp_lv_err_prcntl_all, [lvp_lv_err_prcntl], axis=0)
        lvp_lv_top_rs_all = np.append(lvp_lv_top_rs_all, [lvp_lv_top_rs], axis=0)
        lvp_lv_iter_err_all = np.append(lvp_lv_iter_err_all, [iter_err_df.values], axis=0)
        lvp_lv_iter_rs_all = np.append(lvp_lv_iter_rs_all, [iter_rs_df.values], axis=0)
    
    # lmp + voro + lv => lv
    init_prec = 0.15
    rand_prec = 0.7
    iter_num = 3
    for iter_num in range(in_start, in_end, in_step):
        res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     False, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        if (iter_num == in_start):
            l_v_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            l_v_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            l_v_lv_err_prcntl = np.append(l_v_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            l_v_lv_top_rs = np.append(l_v_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        l_v_lv_err_prcntl_all = np.array([l_v_lv_err_prcntl])
        l_v_lv_top_rs_all = np.array([l_v_lv_top_rs])
        l_v_lv_iter_err_all = np.array([iter_err_df.values])
        l_v_lv_iter_rs_all = np.array([iter_rs_df.values])
    else:
        l_v_lv_err_prcntl_all = np.append(l_v_lv_err_prcntl_all, [l_v_lv_err_prcntl], axis=0)
        l_v_lv_top_rs_all = np.append(l_v_lv_top_rs_all, [l_v_lv_top_rs], axis=0)
        l_v_lv_iter_err_all = np.append(l_v_lv_iter_err_all, [iter_err_df.values], axis=0)
        l_v_lv_iter_rs_all = np.append(l_v_lv_iter_rs_all, [iter_rs_df.values], axis=0)
    
    # hstr: lmp + voro + lv => lv
    init_prec = 0
    rand_prec = 0.3
    iter_num = 1
    for iter_num in range(in_start, in_end, in_step):
        res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     True, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        if (iter_num == in_start):
            h_l_v_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_l_v_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_l_v_lv_err_prcntl = np.append(h_l_v_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_l_v_lv_top_rs = np.append(h_l_v_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_l_v_lv_err_prcntl_all = np.array([h_l_v_lv_err_prcntl])
        h_l_v_lv_top_rs_all = np.array([h_l_v_lv_top_rs])
        h_l_v_lv_iter_err_all = np.array([iter_err_df.values])
        h_l_v_lv_iter_rs_all = np.array([iter_rs_df.values])
    else:
        h_l_v_lv_err_prcntl_all = np.append(h_l_v_lv_err_prcntl_all, [h_l_v_lv_err_prcntl], axis=0)
        h_l_v_lv_top_rs_all = np.append(h_l_v_lv_top_rs_all, [h_l_v_lv_top_rs], axis=0)
        h_l_v_lv_iter_err_all = np.append(h_l_v_lv_iter_err_all, [iter_err_df.values], axis=0)
        h_l_v_lv_iter_rs_all = np.append(h_l_v_lv_iter_rs_all, [iter_rs_df.values], axis=0)

print "For different numbers of iterations:"
colns = map(str, range(in_start, in_end, in_step))

print "lv + lv => lv"
print "Recall score:"
lv_lv_top_rs_avg = np.sum(lv_lv_top_rs_all, axis=0) / len(lv_lv_top_rs_all)
lv_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(lv_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print lv_lv_top_rs_df
df2csv(lv_lv_top_rs_df, dirname + 'top_rs_lv_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
lv_lv_err_prcntl_avg = np.sum(lv_lv_err_prcntl_all, axis=0) / len(lv_lv_err_prcntl_all)
lv_lv_err_prcntl_df = pd.DataFrame(np.transpose(lv_lv_err_prcntl_avg), columns=colns)
print lv_lv_err_prcntl_df
df2csv(lv_lv_err_prcntl_df, dirname + 'err_prcntl_lv_lv_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
lv_lv_iter_rs_avg = np.sum(lv_lv_iter_rs_all, axis=0) / len(lv_lv_iter_rs_all)
lv_lv_iter_rs_df = pd.DataFrame(lv_lv_iter_rs_avg, columns=range(in_end))
print lv_lv_iter_rs_df
df2csv(lv_lv_iter_rs_df, dirname + 'iter_rs_lv_lv_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
lv_lv_iter_err_avg = np.sum(lv_lv_iter_err_all, axis=0) / len(lv_lv_iter_err_all)
lv_lv_iter_err_df = pd.DataFrame(lv_lv_iter_err_avg, columns=range(in_end))
print lv_lv_iter_err_df
df2csv(lv_lv_iter_err_df, dirname + 'iter_err_lv_lv_' + str(smpl_num) + '.csv')

print "lv+ + lv => lv"
print "Recall score:"
lvp_lv_top_rs_avg = np.sum(lvp_lv_top_rs_all, axis=0) / len(lvp_lv_top_rs_all)
lvp_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(lvp_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print lvp_lv_top_rs_df
df2csv(lvp_lv_top_rs_df, dirname + 'top_rs_lvp_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
lvp_lv_err_prcntl_avg = np.sum(lvp_lv_err_prcntl_all, axis=0) / len(lvp_lv_err_prcntl_all)
lvp_lv_err_prcntl_df = pd.DataFrame(np.transpose(lvp_lv_err_prcntl_avg), columns=colns)
print lvp_lv_err_prcntl_df
df2csv(lvp_lv_err_prcntl_df, dirname + 'err_prcntl_lvp_lv_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
lvp_lv_iter_rs_avg = np.sum(lvp_lv_iter_rs_all, axis=0) / len(lvp_lv_iter_rs_all)
lvp_lv_iter_rs_df = pd.DataFrame(lvp_lv_iter_rs_avg, columns=range(in_end))
print lvp_lv_iter_rs_df
df2csv(lvp_lv_iter_rs_df, dirname + 'iter_rs_lvp_lv_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
lvp_lv_iter_err_avg = np.sum(lvp_lv_iter_err_all, axis=0) / len(lvp_lv_iter_err_all)
lvp_lv_iter_err_df = pd.DataFrame(lvp_lv_iter_err_avg, columns=range(in_end))
print lvp_lv_iter_err_df
df2csv(lvp_lv_iter_err_df, dirname + 'iter_err_lvp_lv_' + str(smpl_num) + '.csv')

print "lmp + voro + lv => lv"
print "Recall score:"
l_v_lv_top_rs_avg = np.sum(l_v_lv_top_rs_all, axis=0) / len(l_v_lv_top_rs_all)
l_v_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(l_v_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print l_v_lv_top_rs_df
df2csv(l_v_lv_top_rs_df, dirname + 'top_rs_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
l_v_lv_err_prcntl_avg = np.sum(l_v_lv_err_prcntl_all, axis=0) / len(l_v_lv_err_prcntl_all)
l_v_lv_err_prcntl_df = pd.DataFrame(np.transpose(l_v_lv_err_prcntl_avg), columns=colns)
print l_v_lv_err_prcntl_df
df2csv(l_v_lv_err_prcntl_df, dirname + 'err_prcntl_l_v_lv_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
l_v_lv_iter_rs_avg = np.sum(l_v_lv_iter_rs_all, axis=0) / len(l_v_lv_iter_rs_all)
l_v_lv_iter_rs_df = pd.DataFrame(l_v_lv_iter_rs_avg, columns=range(in_end))
print l_v_lv_iter_rs_df
df2csv(l_v_lv_iter_rs_df, dirname + 'iter_rs_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
l_v_lv_iter_err_avg = np.sum(l_v_lv_iter_err_all, axis=0) / len(l_v_lv_iter_err_all)
l_v_lv_iter_err_df = pd.DataFrame(l_v_lv_iter_err_avg, columns=range(in_end))
print l_v_lv_iter_err_df
df2csv(l_v_lv_iter_err_df, dirname + 'iter_err_l_v_lv_' + str(smpl_num) + '.csv')

print "hstr: lmp + voro + lv => lv"
print "Recall score:"
h_l_v_lv_top_rs_avg = np.sum(h_l_v_lv_top_rs_all, axis=0) / len(h_l_v_lv_top_rs_all)
h_l_v_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_l_v_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_l_v_lv_top_rs_df
df2csv(h_l_v_lv_top_rs_df, dirname + 'top_rs_h_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_l_v_lv_err_prcntl_avg = np.sum(h_l_v_lv_err_prcntl_all, axis=0) / len(h_l_v_lv_err_prcntl_all)
h_l_v_lv_err_prcntl_df = pd.DataFrame(np.transpose(h_l_v_lv_err_prcntl_avg), columns=colns)
print h_l_v_lv_err_prcntl_df
df2csv(h_l_v_lv_err_prcntl_df, dirname + 'err_prcntl_h_l_v_lv_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
h_l_v_lv_iter_rs_avg = np.sum(h_l_v_lv_iter_rs_all, axis=0) / len(h_l_v_lv_iter_rs_all)
h_l_v_lv_iter_rs_df = pd.DataFrame(h_l_v_lv_iter_rs_avg, columns=range(in_end))
print h_l_v_lv_iter_rs_df
df2csv(h_l_v_lv_iter_rs_df, dirname + 'iter_rs_h_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
h_l_v_lv_iter_err_avg = np.sum(h_l_v_lv_iter_err_all, axis=0) / len(h_l_v_lv_iter_err_all)
h_l_v_lv_iter_err_df = pd.DataFrame(h_l_v_lv_iter_err_avg, columns=range(in_end))
print h_l_v_lv_iter_err_df
df2csv(h_l_v_lv_iter_err_df, dirname + 'iter_err_h_l_v_lv_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/ht-sw/iteration/' + perf_coln + '/'

in_start = 1
in_end = 7
in_step = 1

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    ht_df = ht_df.sort_values(ht_conf_colns).reset_index(drop=True)
    sw_df = sw_df.sort_values(sw_conf_colns).reset_index(drop=True)
    hs_df = hs_df.sort_values(hs_conf_colns).reset_index(drop=True)
    hs_in_df = hs_in_df.sort_values(hs_in_conf_colns).reset_index(drop=True)

    ht_df = ht_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    sw_df = sw_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_df = hs_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_in_df = hs_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # hs + hs => hs
    rand_prec = 0.4
    iter_num = 5
    for iter_num in range(in_start, in_end, in_step):
        res = sa_sa(hs_df, hs_conf_colns, perf_coln, smpl_num, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        rows = top_rs_df[['top_prec', 'top_num']].values
        if (iter_num == in_start):
            hs_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            hs_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            hs_hs_err_prcntl = np.append(hs_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            hs_hs_top_rs = np.append(hs_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)

    if (rand_seed == rs_start):
        hs_hs_err_prcntl_all = np.array([hs_hs_err_prcntl])
        hs_hs_top_rs_all = np.array([hs_hs_top_rs])
        hs_hs_iter_err_all = np.array([iter_err_df.values])
        hs_hs_iter_rs_all = np.array([iter_rs_df.values])
    else:
        hs_hs_err_prcntl_all = np.append(hs_hs_err_prcntl_all, [hs_hs_err_prcntl], axis=0)
        hs_hs_top_rs_all = np.append(hs_hs_top_rs_all, [hs_hs_top_rs], axis=0)
        hs_hs_iter_err_all = np.append(hs_hs_iter_err_all, [iter_err_df.values], axis=0)
        hs_hs_iter_rs_all = np.append(hs_hs_iter_rs_all, [iter_rs_df.values], axis=0)
    
    # hs+ + hs => hs
    init_prec = 0.05
    rand_prec = 0.8
    iter_num = 2
    for iter_num in range(in_start, in_end, in_step):
        res = sap_sa(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        if (iter_num == in_start):
            hsp_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            hsp_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            hsp_hs_err_prcntl = np.append(hsp_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            hsp_hs_top_rs = np.append(hsp_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        hsp_hs_err_prcntl_all = np.array([hsp_hs_err_prcntl])
        hsp_hs_top_rs_all = np.array([hsp_hs_top_rs])
        hsp_hs_iter_err_all = np.array([iter_err_df.values])
        hsp_hs_iter_rs_all = np.array([iter_rs_df.values])
    else:
        hsp_hs_err_prcntl_all = np.append(hsp_hs_err_prcntl_all, [hsp_hs_err_prcntl], axis=0)
        hsp_hs_top_rs_all = np.append(hsp_hs_top_rs_all, [hsp_hs_top_rs], axis=0)
        hsp_hs_iter_err_all = np.append(hsp_hs_iter_err_all, [iter_err_df.values], axis=0)
        hsp_hs_iter_rs_all = np.append(hsp_hs_iter_rs_all, [iter_rs_df.values], axis=0)
    
    # ht + sw + hs => hs
    init_prec = 0.15
    rand_prec = 0.3
    iter_num = 1
    for iter_num in range(in_start, in_end, in_step):
        res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     False, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        if (iter_num == in_start):
            h_s_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_s_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_s_hs_err_prcntl = np.append(h_s_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_s_hs_top_rs = np.append(h_s_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_s_hs_err_prcntl_all = np.array([h_s_hs_err_prcntl])
        h_s_hs_top_rs_all = np.array([h_s_hs_top_rs])
        h_s_hs_iter_err_all = np.array([iter_err_df.values])
        h_s_hs_iter_rs_all = np.array([iter_rs_df.values])
    else:
        h_s_hs_err_prcntl_all = np.append(h_s_hs_err_prcntl_all, [h_s_hs_err_prcntl], axis=0)
        h_s_hs_top_rs_all = np.append(h_s_hs_top_rs_all, [h_s_hs_top_rs], axis=0)
        h_s_hs_iter_err_all = np.append(h_s_hs_iter_err_all, [iter_err_df.values], axis=0)
        h_s_hs_iter_rs_all = np.append(h_s_hs_iter_rs_all, [iter_rs_df.values], axis=0)
    
    # hstr: ht + sw + hs => hs
    init_prec = 0
    rand_prec = 0.3
    iter_num = 1
    for iter_num in range(in_start, in_end, in_step):
        res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     True, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df, iter_err_df, iter_rs_df = res[0], res[1], res[3], res[4]
        if (iter_num == in_start):
            h_h_s_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_h_s_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_h_s_hs_err_prcntl = np.append(h_h_s_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_h_s_hs_top_rs = np.append(h_h_s_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_h_s_hs_err_prcntl_all = np.array([h_h_s_hs_err_prcntl])
        h_h_s_hs_top_rs_all = np.array([h_h_s_hs_top_rs])
        h_h_s_hs_iter_err_all = np.array([iter_err_df.values])
        h_h_s_hs_iter_rs_all = np.array([iter_rs_df.values])
    else:
        h_h_s_hs_err_prcntl_all = np.append(h_h_s_hs_err_prcntl_all, [h_h_s_hs_err_prcntl], axis=0)
        h_h_s_hs_top_rs_all = np.append(h_h_s_hs_top_rs_all, [h_h_s_hs_top_rs], axis=0)
        h_h_s_hs_iter_err_all = np.append(h_h_s_hs_iter_err_all, [iter_err_df.values], axis=0)
        h_h_s_hs_iter_rs_all = np.append(h_h_s_hs_iter_rs_all, [iter_rs_df.values], axis=0)

print "For different numbers of iterations:"
colns = map(str, range(in_start, in_end, in_step))

print "hs + hs => hs"
print "Recall score:"
hs_hs_top_rs_avg = np.sum(hs_hs_top_rs_all, axis=0) / len(hs_hs_top_rs_all)
hs_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(hs_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print hs_hs_top_rs_df
df2csv(hs_hs_top_rs_df, dirname + 'top_rs_hs_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
hs_hs_err_prcntl_avg = np.sum(hs_hs_err_prcntl_all, axis=0) / len(hs_hs_err_prcntl_all)
hs_hs_err_prcntl_df = pd.DataFrame(np.transpose(hs_hs_err_prcntl_avg), columns=colns)
print hs_hs_err_prcntl_df
df2csv(hs_hs_err_prcntl_df, dirname + 'err_prcntl_hs_hs_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
hs_hs_iter_rs_avg = np.sum(hs_hs_iter_rs_all, axis=0) / len(hs_hs_iter_rs_all)
hs_hs_iter_rs_df = pd.DataFrame(hs_hs_iter_rs_avg, columns=range(in_end))
print hs_hs_iter_rs_df
df2csv(hs_hs_iter_rs_df, dirname + 'iter_rs_hs_hs_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
hs_hs_iter_err_avg = np.sum(hs_hs_iter_err_all, axis=0) / len(hs_hs_iter_err_all)
hs_hs_iter_err_df = pd.DataFrame(hs_hs_iter_err_avg, columns=range(in_end))
print hs_hs_iter_err_df
df2csv(hs_hs_iter_err_df, dirname + 'iter_err_hs_hs_' + str(smpl_num) + '.csv')

print "hs+ + hs => hs"
print "Recall score:"
hsp_hs_top_rs_avg = np.sum(hsp_hs_top_rs_all, axis=0) / len(hsp_hs_top_rs_all)
hsp_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(hsp_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print hsp_hs_top_rs_df
df2csv(hsp_hs_top_rs_df, dirname + 'top_rs_hsp_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
hsp_hs_err_prcntl_avg = np.sum(hsp_hs_err_prcntl_all, axis=0) / len(hsp_hs_err_prcntl_all)
hsp_hs_err_prcntl_df = pd.DataFrame(np.transpose(hsp_hs_err_prcntl_avg), columns=colns)
print hsp_hs_err_prcntl_df
df2csv(hsp_hs_err_prcntl_df, dirname + 'err_prcntl_hsp_hs_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
hsp_hs_iter_rs_avg = np.sum(hsp_hs_iter_rs_all, axis=0) / len(hsp_hs_iter_rs_all)
hsp_hs_iter_rs_df = pd.DataFrame(hsp_hs_iter_rs_avg, columns=range(in_end))
print hsp_hs_iter_rs_df
df2csv(hsp_hs_iter_rs_df, dirname + 'iter_rs_hsp_hs_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
hsp_hs_iter_err_avg = np.sum(hsp_hs_iter_err_all, axis=0) / len(hsp_hs_iter_err_all)
hsp_hs_iter_err_df = pd.DataFrame(hsp_hs_iter_err_avg, columns=range(in_end))
print hsp_hs_iter_err_df
df2csv(hsp_hs_iter_err_df, dirname + 'iter_err_hsp_hs_' + str(smpl_num) + '.csv')

print "ht + sw + hs => hs"
print "Recall score:"
h_s_hs_top_rs_avg = np.sum(h_s_hs_top_rs_all, axis=0) / len(h_s_hs_top_rs_all)
h_s_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_s_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_s_hs_top_rs_df
df2csv(h_s_hs_top_rs_df, dirname + 'top_rs_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_s_hs_err_prcntl_avg = np.sum(h_s_hs_err_prcntl_all, axis=0) / len(h_s_hs_err_prcntl_all)
h_s_hs_err_prcntl_df = pd.DataFrame(np.transpose(h_s_hs_err_prcntl_avg), columns=colns)
print h_s_hs_err_prcntl_df
df2csv(h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_s_hs_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
h_s_hs_iter_rs_avg = np.sum(h_s_hs_iter_rs_all, axis=0) / len(h_s_hs_iter_rs_all)
h_s_hs_iter_rs_df = pd.DataFrame(h_s_hs_iter_rs_avg, columns=range(in_end))
print h_s_hs_iter_rs_df
df2csv(h_s_hs_iter_rs_df, dirname + 'iter_rs_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
h_s_hs_iter_err_avg = np.sum(h_s_hs_iter_err_all, axis=0) / len(h_s_hs_iter_err_all)
h_s_hs_iter_err_df = pd.DataFrame(h_s_hs_iter_err_avg, columns=range(in_end))
print h_s_hs_iter_err_df
df2csv(h_s_hs_iter_err_df, dirname + 'iter_err_h_s_hs_' + str(smpl_num) + '.csv')

print "hstr: ht + sw + hs => hs"
print "Recall score:"
h_h_s_hs_top_rs_avg = np.sum(h_h_s_hs_top_rs_all, axis=0) / len(h_h_s_hs_top_rs_all)
h_h_s_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_h_s_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_h_s_hs_top_rs_df
df2csv(h_h_s_hs_top_rs_df, dirname + 'top_rs_h_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_h_s_hs_err_prcntl_avg = np.sum(h_h_s_hs_err_prcntl_all, axis=0) / len(h_h_s_hs_err_prcntl_all)
h_h_s_hs_err_prcntl_df = pd.DataFrame(np.transpose(h_h_s_hs_err_prcntl_avg), columns=colns)
print h_h_s_hs_err_prcntl_df
df2csv(h_h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_h_s_hs_' + str(smpl_num) + '.csv')
print "Recall score per iteration:"
h_h_s_hs_iter_rs_avg = np.sum(h_h_s_hs_iter_rs_all, axis=0) / len(h_h_s_hs_iter_rs_all)
h_h_s_hs_iter_rs_df = pd.DataFrame(h_h_s_hs_iter_rs_avg, columns=range(in_end))
print h_h_s_hs_iter_rs_df
df2csv(h_h_s_hs_iter_rs_df, dirname + 'iter_rs_h_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error per iteration:"
h_h_s_hs_iter_err_avg = np.sum(h_h_s_hs_iter_err_all, axis=0) / len(h_h_s_hs_iter_err_all)
h_h_s_hs_iter_err_df = pd.DataFrame(h_h_s_hs_iter_err_avg, columns=range(in_end))
print h_h_s_hs_iter_err_df
df2csv(h_h_s_hs_iter_err_df, dirname + 'iter_err_h_h_s_hs_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/lmp-voro/rand_prec/' + perf_coln + '/'

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    lmp_df = lmp_df.sort_values(lmp_conf_colns).reset_index(drop=True)
    voro_df = voro_df.sort_values(voro_conf_colns).reset_index(drop=True)
    lv_df = lv_df.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_in_df = lv_in_df.sort_values(lv_in_conf_colns).reset_index(drop=True)

    lmp_df = lmp_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    voro_df = voro_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_df = lv_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_in_df = lv_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # lv + lv => lv
    rand_prec = 0.3
    iter_num = 4
    rp_start = 0.05
    rp_end = 0.96
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = sa_sa(lv_df, lv_conf_colns, perf_coln, smpl_num, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        rows = top_rs_df[['top_prec', 'top_num']].values
        if (rand_prec == rp_start):
            lv_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            lv_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            lv_lv_err_prcntl = np.append(lv_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            lv_lv_top_rs = np.append(lv_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        lv_lv_err_prcntl_all = np.array([lv_lv_err_prcntl])
        lv_lv_top_rs_all = np.array([lv_lv_top_rs])
    else:
        lv_lv_err_prcntl_all = np.append(lv_lv_err_prcntl_all, [lv_lv_err_prcntl], axis=0)
        lv_lv_top_rs_all = np.append(lv_lv_top_rs_all, [lv_lv_top_rs], axis=0)
        
    # lv+ + lv => lv
    init_prec = 0.1
    rand_prec = 0.5
    iter_num = 2
    rp_start = 0.05
    rp_end = 0.86
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = sap_sa(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (rand_prec == rp_start):
            lvp_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            lvp_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            lvp_lv_err_prcntl = np.append(lvp_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            lvp_lv_top_rs = np.append(lvp_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        lvp_lv_err_prcntl_all = np.array([lvp_lv_err_prcntl])
        lvp_lv_top_rs_all = np.array([lvp_lv_top_rs])
    else:
        lvp_lv_err_prcntl_all = np.append(lvp_lv_err_prcntl_all, [lvp_lv_err_prcntl], axis=0)
        lvp_lv_top_rs_all = np.append(lvp_lv_top_rs_all, [lvp_lv_top_rs], axis=0)
    
    # lmp + voro + lv => lv
    init_prec = 0.15
    rand_prec = 0.7
    iter_num = 3
    rp_start = 0.05
    rp_end = 0.81
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     False, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (rand_prec == rp_start):
            l_v_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            l_v_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            l_v_lv_err_prcntl = np.append(l_v_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            l_v_lv_top_rs = np.append(l_v_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        l_v_lv_err_prcntl_all = np.array([l_v_lv_err_prcntl])
        l_v_lv_top_rs_all = np.array([l_v_lv_top_rs])
    else:
        l_v_lv_err_prcntl_all = np.append(l_v_lv_err_prcntl_all, [l_v_lv_err_prcntl], axis=0)
        l_v_lv_top_rs_all = np.append(l_v_lv_top_rs_all, [l_v_lv_top_rs], axis=0)
    
    # hstr: lmp + voro + lv => lv
    rand_prec = 0.3
    iter_num = 1
    rp_start = 0.05
    rp_end = 0.96
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     True, 0, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (rand_prec == rp_start):
            h_l_v_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_l_v_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_l_v_lv_err_prcntl = np.append(h_l_v_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_l_v_lv_top_rs = np.append(h_l_v_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_l_v_lv_err_prcntl_all = np.array([h_l_v_lv_err_prcntl])
        h_l_v_lv_top_rs_all = np.array([h_l_v_lv_top_rs])
    else:
        h_l_v_lv_err_prcntl_all = np.append(h_l_v_lv_err_prcntl_all, [h_l_v_lv_err_prcntl], axis=0)
        h_l_v_lv_top_rs_all = np.append(h_l_v_lv_top_rs_all, [h_l_v_lv_top_rs], axis=0)
    
print "For different random sample precentages:"
print "lv + lv => lv"
colns = map(str, range(5, 96, 5))
print "Recall score:"
lv_lv_top_rs_avg = np.sum(lv_lv_top_rs_all, axis=0) / len(lv_lv_top_rs_all)
lv_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(lv_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print lv_lv_top_rs_df
df2csv(lv_lv_top_rs_df, dirname + 'top_rs_lv_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
lv_lv_err_prcntl_avg = np.sum(lv_lv_err_prcntl_all, axis=0) / len(lv_lv_err_prcntl_all)
lv_lv_err_prcntl_df = pd.DataFrame(np.transpose(lv_lv_err_prcntl_avg), columns=colns)
print lv_lv_err_prcntl_df
df2csv(lv_lv_err_prcntl_df, dirname + 'err_prcntl_lv_lv_' + str(smpl_num) + '.csv')

print "lv+ + lv => lv"
colns = map(str, range(5, 86, 5))
print "Recall score:"
lvp_lv_top_rs_avg = np.sum(lvp_lv_top_rs_all, axis=0) / len(lvp_lv_top_rs_all)
lvp_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(lvp_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print lvp_lv_top_rs_df
df2csv(lvp_lv_top_rs_df, dirname + 'top_rs_lvp_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
lvp_lv_err_prcntl_avg = np.sum(lvp_lv_err_prcntl_all, axis=0) / len(lvp_lv_err_prcntl_all)
lvp_lv_err_prcntl_df = pd.DataFrame(np.transpose(lvp_lv_err_prcntl_avg), columns=colns)
print lvp_lv_err_prcntl_df
df2csv(lvp_lv_err_prcntl_df, dirname + 'err_prcntl_lvp_lv_' + str(smpl_num) + '.csv')

print "lmp + voro + lv => lv"
colns = map(str, range(5, 81, 5))
print "Recall score:"
l_v_lv_top_rs_avg = np.sum(l_v_lv_top_rs_all, axis=0) / len(l_v_lv_top_rs_all)
l_v_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(l_v_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print l_v_lv_top_rs_df
df2csv(l_v_lv_top_rs_df, dirname + 'top_rs_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
l_v_lv_err_prcntl_avg = np.sum(l_v_lv_err_prcntl_all, axis=0) / len(l_v_lv_err_prcntl_all)
l_v_lv_err_prcntl_df = pd.DataFrame(np.transpose(l_v_lv_err_prcntl_avg), columns=colns)
print l_v_lv_err_prcntl_df
df2csv(l_v_lv_err_prcntl_df, dirname + 'err_prcntl_l_v_lv_' + str(smpl_num) + '.csv')

print "hstr: lmp + voro + lv => lv"
colns = map(str, range(5, 96, 5))
print "Recall score:"
h_l_v_lv_top_rs_avg = np.sum(h_l_v_lv_top_rs_all, axis=0) / len(h_l_v_lv_top_rs_all)
h_l_v_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_l_v_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_l_v_lv_top_rs_df
df2csv(h_l_v_lv_top_rs_df, dirname + 'top_rs_h_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_l_v_lv_err_prcntl_avg = np.sum(h_l_v_lv_err_prcntl_all, axis=0) / len(h_l_v_lv_err_prcntl_all)
h_l_v_lv_err_prcntl_df = pd.DataFrame(np.transpose(h_l_v_lv_err_prcntl_avg), columns=colns)
print h_l_v_lv_err_prcntl_df
df2csv(h_l_v_lv_err_prcntl_df, dirname + 'err_prcntl_h_l_v_lv_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/ht-sw/rand_prec/' + perf_coln + '/'

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    ht_df = ht_df.sort_values(ht_conf_colns).reset_index(drop=True)
    sw_df = sw_df.sort_values(sw_conf_colns).reset_index(drop=True)
    hs_df = hs_df.sort_values(hs_conf_colns).reset_index(drop=True)
    hs_in_df = hs_in_df.sort_values(hs_in_conf_colns).reset_index(drop=True)

    ht_df = ht_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    sw_df = sw_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_df = hs_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_in_df = hs_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

    # hs + hs => hs
    rand_prec = 0.4
    iter_num = 5
    rp_start = 0.05
    rp_end = 0.96
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = sa_sa(hs_df, hs_conf_colns, perf_coln, smpl_num, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        rows = top_rs_df[['top_prec', 'top_num']].values
        if (rand_prec == rp_start):
            hs_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            hs_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            hs_hs_err_prcntl = np.append(hs_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            hs_hs_top_rs = np.append(hs_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        hs_hs_err_prcntl_all = np.array([hs_hs_err_prcntl])
        hs_hs_top_rs_all = np.array([hs_hs_top_rs])
    else:
        hs_hs_err_prcntl_all = np.append(hs_hs_err_prcntl_all, [hs_hs_err_prcntl], axis=0)
        hs_hs_top_rs_all = np.append(hs_hs_top_rs_all, [hs_hs_top_rs], axis=0)
        
    # hs+ + hs => hs
    init_prec = 0.05
    rand_prec = 0.25
    iter_num = 2
    rp_start = 0.05
    rp_end = 0.91
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = sap_sa(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (rand_prec == rp_start):
            hsp_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            hsp_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            hsp_hs_err_prcntl = np.append(hsp_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            hsp_hs_top_rs = np.append(hsp_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        hsp_hs_err_prcntl_all = np.array([hsp_hs_err_prcntl])
        hsp_hs_top_rs_all = np.array([hsp_hs_top_rs])
    else:
        hsp_hs_err_prcntl_all = np.append(hsp_hs_err_prcntl_all, [hsp_hs_err_prcntl], axis=0)
        hsp_hs_top_rs_all = np.append(hsp_hs_top_rs_all, [hsp_hs_top_rs], axis=0)
    
    # ht + sw + hs => hs
    init_prec = 0.15
    rand_prec = 0.3
    iter_num = 1
    rp_start = 0.05
    rp_end = 0.81
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     False, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (rand_prec == rp_start):
            h_s_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_s_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_s_hs_err_prcntl = np.append(h_s_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_s_hs_top_rs = np.append(h_s_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_s_hs_err_prcntl_all = np.array([h_s_hs_err_prcntl])
        h_s_hs_top_rs_all = np.array([h_s_hs_top_rs])
    else:
        h_s_hs_err_prcntl_all = np.append(h_s_hs_err_prcntl_all, [h_s_hs_err_prcntl], axis=0)
        h_s_hs_top_rs_all = np.append(h_s_hs_top_rs_all, [h_s_hs_top_rs], axis=0)

    # hstr: ht + sw + hs => hs
    rand_prec = 0.3
    iter_num = 1
    rp_start = 0.05
    rp_end = 0.96
    rp_step = 0.05
    for rand_prec in np.arange(rp_start, rp_end, rp_step):
        res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     True, 0, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (rand_prec == rp_start):
            h_h_s_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_h_s_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_h_s_hs_err_prcntl = np.append(h_h_s_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_h_s_hs_top_rs = np.append(h_h_s_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_h_s_hs_err_prcntl_all = np.array([h_h_s_hs_err_prcntl])
        h_h_s_hs_top_rs_all = np.array([h_h_s_hs_top_rs])
    else:
        h_h_s_hs_err_prcntl_all = np.append(h_h_s_hs_err_prcntl_all, [h_h_s_hs_err_prcntl], axis=0)
        h_h_s_hs_top_rs_all = np.append(h_h_s_hs_top_rs_all, [h_h_s_hs_top_rs], axis=0)
    
print "For different random sample precentages:"
print "hs + hs => hs"
colns = map(str, range(5, 96, 5))
print "Recall score:"
hs_hs_top_rs_avg = np.sum(hs_hs_top_rs_all, axis=0) / len(hs_hs_top_rs_all)
hs_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(hs_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print hs_hs_top_rs_df
df2csv(hs_hs_top_rs_df, dirname + 'top_rs_hs_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
hs_hs_err_prcntl_avg = np.sum(hs_hs_err_prcntl_all, axis=0) / len(hs_hs_err_prcntl_all)
hs_hs_err_prcntl_df = pd.DataFrame(np.transpose(hs_hs_err_prcntl_avg), columns=colns)
print hs_hs_err_prcntl_df
df2csv(hs_hs_err_prcntl_df, dirname + 'err_prcntl_hs_hs_' + str(smpl_num) + '.csv')

print "hs+ + hs => hs"
colns = map(str, range(5, 91, 5))
print "Recall score:"
hsp_hs_top_rs_avg = np.sum(hsp_hs_top_rs_all, axis=0) / len(hsp_hs_top_rs_all)
hsp_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(hsp_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print hsp_hs_top_rs_df
df2csv(hsp_hs_top_rs_df, dirname + 'top_rs_hsp_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
hsp_hs_err_prcntl_avg = np.sum(hsp_hs_err_prcntl_all, axis=0) / len(hsp_hs_err_prcntl_all)
hsp_hs_err_prcntl_df = pd.DataFrame(np.transpose(hsp_hs_err_prcntl_avg), columns=colns)
print hsp_hs_err_prcntl_df
df2csv(hsp_hs_err_prcntl_df, dirname + 'err_prcntl_hsp_hs_' + str(smpl_num) + '.csv')

print "ht + sw + hs => hs"
colns = map(str, range(5, 81, 5))
print "Recall score:"
h_s_hs_top_rs_avg = np.sum(h_s_hs_top_rs_all, axis=0) / len(h_s_hs_top_rs_all)
h_s_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_s_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_s_hs_top_rs_df
df2csv(h_s_hs_top_rs_df, dirname + 'top_rs_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_s_hs_err_prcntl_avg = np.sum(h_s_hs_err_prcntl_all, axis=0) / len(h_s_hs_err_prcntl_all)
h_s_hs_err_prcntl_df = pd.DataFrame(np.transpose(h_s_hs_err_prcntl_avg), columns=colns)
print h_s_hs_err_prcntl_df
df2csv(h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_s_hs_' + str(smpl_num) + '.csv')

print "hstr: ht + sw + hs => hs"
colns = map(str, range(5, 96, 5))
print "Recall score:"
h_h_s_hs_top_rs_avg = np.sum(h_h_s_hs_top_rs_all, axis=0) / len(h_h_s_hs_top_rs_all)
h_h_s_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_h_s_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_h_s_hs_top_rs_df
df2csv(h_h_s_hs_top_rs_df, dirname + 'top_rs_h_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_h_s_hs_err_prcntl_avg = np.sum(h_h_s_hs_err_prcntl_all, axis=0) / len(h_h_s_hs_err_prcntl_all)
h_h_s_hs_err_prcntl_df = pd.DataFrame(np.transpose(h_h_s_hs_err_prcntl_avg), columns=colns)
print h_h_s_hs_err_prcntl_df
df2csv(h_h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_h_s_hs_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/lmp-voro/init_prec/' + perf_coln + '/'

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    lmp_df = lmp_df.sort_values(lmp_conf_colns).reset_index(drop=True)
    voro_df = voro_df.sort_values(voro_conf_colns).reset_index(drop=True)
    lv_df = lv_df.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_in_df = lv_in_df.sort_values(lv_in_conf_colns).reset_index(drop=True)

    lmp_df = lmp_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    voro_df = voro_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_df = lv_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_in_df = lv_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # lv+ + lv => lv
    init_prec = 0.1
    rand_prec = 0.5
    iter_num = 2
    ip_start = 0.05
    ip_end = 0.46
    ip_step = 0.05
    for init_prec in np.arange(ip_start, ip_end, ip_step):
        res = sap_sa(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        rows = top_rs_df[['top_prec', 'top_num']].values
        if (init_prec == ip_start):
            lvp_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            lvp_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            lvp_lv_err_prcntl = np.append(lvp_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            lvp_lv_top_rs = np.append(lvp_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        lvp_lv_err_prcntl_all = np.array([lvp_lv_err_prcntl])
        lvp_lv_top_rs_all = np.array([lvp_lv_top_rs])
    else:
        lvp_lv_err_prcntl_all = np.append(lvp_lv_err_prcntl_all, [lvp_lv_err_prcntl], axis=0)
        lvp_lv_top_rs_all = np.append(lvp_lv_top_rs_all, [lvp_lv_top_rs], axis=0)
    
    # lmp + voro + lv => lv
    init_prec = 0.15
    rand_prec = 0.7
    iter_num = 3
    ip_start = 0.05
    ip_end = 0.26
    ip_step = 0.05
    for init_prec in np.arange(ip_start, ip_end, ip_step):
        res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, \
                     False, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (init_prec == ip_start):
            l_v_lv_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            l_v_lv_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            l_v_lv_err_prcntl = np.append(l_v_lv_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            l_v_lv_top_rs = np.append(l_v_lv_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        l_v_lv_err_prcntl_all = np.array([l_v_lv_err_prcntl])
        l_v_lv_top_rs_all = np.array([l_v_lv_top_rs])
    else:
        l_v_lv_err_prcntl_all = np.append(l_v_lv_err_prcntl_all, [l_v_lv_err_prcntl], axis=0)
        l_v_lv_top_rs_all = np.append(l_v_lv_top_rs_all, [l_v_lv_top_rs], axis=0)
    
print "For different initial (transferred) sample precentages:"
print "lv+ + lv => lv"
colns = map(str, range(5, 46, 5))
print "Recall score:"
lvp_lv_top_rs_avg = np.sum(lvp_lv_top_rs_all, axis=0) / len(lvp_lv_top_rs_all)
lvp_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(lvp_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print lvp_lv_top_rs_df
df2csv(lvp_lv_top_rs_df, dirname + 'top_rs_lvp_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
lvp_lv_err_prcntl_avg = np.sum(lvp_lv_err_prcntl_all, axis=0) / len(lvp_lv_err_prcntl_all)
lvp_lv_err_prcntl_df = pd.DataFrame(np.transpose(lvp_lv_err_prcntl_avg), columns=colns)
print lvp_lv_err_prcntl_df
df2csv(lvp_lv_err_prcntl_df, dirname + 'err_prcntl_lvp_lv_' + str(smpl_num) + '.csv')

print "lmp + voro + lv => lv"
colns = map(str, range(5, 26, 5))
print "Recall score:"
l_v_lv_top_rs_avg = np.sum(l_v_lv_top_rs_all, axis=0) / len(l_v_lv_top_rs_all)
l_v_lv_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(l_v_lv_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print l_v_lv_top_rs_df
df2csv(l_v_lv_top_rs_df, dirname + 'top_rs_l_v_lv_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
l_v_lv_err_prcntl_avg = np.sum(l_v_lv_err_prcntl_all, axis=0) / len(l_v_lv_err_prcntl_all)
l_v_lv_err_prcntl_df = pd.DataFrame(np.transpose(l_v_lv_err_prcntl_avg), columns=colns)
print l_v_lv_err_prcntl_df
df2csv(l_v_lv_err_prcntl_df, dirname + 'err_prcntl_l_v_lv_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/ht-sw/init_prec/' + perf_coln + '/'

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    ht_df = ht_df.sort_values(ht_conf_colns).reset_index(drop=True)
    sw_df = sw_df.sort_values(sw_conf_colns).reset_index(drop=True)
    hs_df = hs_df.sort_values(hs_conf_colns).reset_index(drop=True)
    hs_in_df = hs_in_df.sort_values(hs_in_conf_colns).reset_index(drop=True)

    ht_df = ht_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    sw_df = sw_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_df = hs_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_in_df = hs_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # hs+ + hs => hs
    init_prec = 0.05
    rand_prec = 0.8
    iter_num = 2
    ip_start = 0.05
    ip_end = 0.16
    ip_step = 0.05
    for init_prec in np.arange(ip_start, ip_end, ip_step):
        res = sap_sa(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        rows = top_rs_df[['top_prec', 'top_num']].values
        if (init_prec == ip_start):
            hsp_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            hsp_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            hsp_hs_err_prcntl = np.append(hsp_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            hsp_hs_top_rs = np.append(hsp_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        hsp_hs_err_prcntl_all = np.array([hsp_hs_err_prcntl])
        hsp_hs_top_rs_all = np.array([hsp_hs_top_rs])
    else:
        hsp_hs_err_prcntl_all = np.append(hsp_hs_err_prcntl_all, [hsp_hs_err_prcntl], axis=0)
        hsp_hs_top_rs_all = np.append(hsp_hs_top_rs_all, [hsp_hs_top_rs], axis=0)
    
    # ht + sw + hs => hs
    init_prec = 0.15
    rand_prec = 0.3
    iter_num = 1
    ip_start = 0.05
    ip_end = 0.66
    ip_step = 0.05
    for init_prec in np.arange(ip_start, ip_end, ip_step):
        res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                     False, init_prec, rand_prec, iter_num)
        top_rs_df, err_prcntl_df = res[0], res[1]
        if (init_prec == ip_start):
            h_s_hs_err_prcntl = np.array([err_prcntl_df[50].loc[range(4)].values])
            h_s_hs_top_rs = np.array([top_rs_df['recall_score'].values])
        else:
            h_s_hs_err_prcntl = np.append(h_s_hs_err_prcntl, [err_prcntl_df[50].loc[range(4)].values], axis=0)
            h_s_hs_top_rs = np.append(h_s_hs_top_rs, [top_rs_df['recall_score'].values], axis=0)
    
    if (rand_seed == rs_start):
        h_s_hs_err_prcntl_all = np.array([h_s_hs_err_prcntl])
        h_s_hs_top_rs_all = np.array([h_s_hs_top_rs])
    else:
        h_s_hs_err_prcntl_all = np.append(h_s_hs_err_prcntl_all, [h_s_hs_err_prcntl], axis=0)
        h_s_hs_top_rs_all = np.append(h_s_hs_top_rs_all, [h_s_hs_top_rs], axis=0)
    
print "For different initial (transferred) sample precentages:"
print "hs+ + hs => hs"
colns = map(str, range(5, 71, 5))
print "Recall score:"
hsp_hs_top_rs_avg = np.sum(hsp_hs_top_rs_all, axis=0) / len(hsp_hs_top_rs_all)
hsp_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(hsp_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print hsp_hs_top_rs_df
df2csv(hsp_hs_top_rs_df, dirname + 'top_rs_hsp_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
hsp_hs_err_prcntl_avg = np.sum(hsp_hs_err_prcntl_all, axis=0) / len(hsp_hs_err_prcntl_all)
hsp_hs_err_prcntl_df = pd.DataFrame(np.transpose(hsp_hs_err_prcntl_avg), columns=colns)
print hsp_hs_err_prcntl_df
df2csv(hsp_hs_err_prcntl_df, dirname + 'err_prcntl_hsp_hs_' + str(smpl_num) + '.csv')

print "ht + sw + hs => hs"
colns = map(str, range(5, 66, 5))
print "Recall score:"
h_s_hs_top_rs_avg = np.sum(h_s_hs_top_rs_all, axis=0) / len(h_s_hs_top_rs_all)
h_s_hs_top_rs_df = pd.DataFrame(np.c_[rows, np.transpose(h_s_hs_top_rs_avg)], columns=['top_prec', 'top_num']+colns)
print h_s_hs_top_rs_df
df2csv(h_s_hs_top_rs_df, dirname + 'top_rs_h_s_hs_' + str(smpl_num) + '.csv')
print "Prediction Error precentile:"
h_s_hs_err_prcntl_avg = np.sum(h_s_hs_err_prcntl_all, axis=0) / len(h_s_hs_err_prcntl_all)
h_s_hs_err_prcntl_df = pd.DataFrame(np.transpose(h_s_hs_err_prcntl_avg), columns=colns)
print h_s_hs_err_prcntl_df
df2csv(h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_s_hs_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/lmp-voro/' + perf_coln + '/'
print gen_top_df(lv_df, perf_coln, 1)

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    lmp_df = lmp_df.sort_values(lmp_conf_colns).reset_index(drop=True)
    voro_df = voro_df.sort_values(voro_conf_colns).reset_index(drop=True)
    lv_df = lv_df.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_in_df = lv_in_df.sort_values(lv_in_conf_colns).reset_index(drop=True)
    lv_shrt_df1 = lv_shrt_df1.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_shrt_df2 = lv_shrt_df2.sort_values(lv_conf_colns).reset_index(drop=True)

    lmp_df = lmp_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    voro_df = voro_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_df = lv_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_in_df = lv_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_shrt_df1 = lv_shrt_df1.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_shrt_df2 = lv_shrt_df2.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # cmbn: lv+ + lv => lv
    res = cmbn_sap(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_lvp_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_lvp_top_mr = np.array([top_rs_df.values])
        cmbn_lvp_top_perf = top_perf
    else:
        cmbn_lvp_err_prcntl = np.append(cmbn_lvp_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_lvp_top_mr = np.append(cmbn_lvp_top_mr, [top_rs_df.values], axis=0)
        cmbn_lvp_top_perf += top_perf
    
    # cmbn: lmp + voro + lv => lv
    res = cmbn_s_a(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_l_v_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_l_v_top_mr = np.array([top_rs_df.values])
        cmbn_l_v_top_perf = top_perf
    else:
        cmbn_l_v_err_prcntl = np.append(cmbn_l_v_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_l_v_top_mr = np.append(cmbn_l_v_top_mr, [top_rs_df.values], axis=0)
        cmbn_l_v_top_perf += top_perf
    
    # drct: lv => lv
    res = drct_sa(lv_df, lv_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        lv_err_prcntl = np.array([err_prcntl_df.values])
        lv_top_mr = np.array([top_rs_df.values])
        lv_top_perf = top_perf
    else:
        lv_err_prcntl = np.append(lv_err_prcntl, [err_prcntl_df.values], axis=0)
        lv_top_mr = np.append(lv_top_mr, [top_rs_df.values], axis=0)
        lv_top_perf += top_perf
    
    # lv + lv => lv
    res = sa_sa(lv_df, lv_conf_colns, perf_coln, smpl_num, 0.3, 4)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        lv_lv_err_prcntl = np.array([err_prcntl_df.values])
        lv_lv_top_mr = np.array([top_rs_df.values])
        lv_lv_top_perf = top_perf
    else:
        lv_lv_err_prcntl = np.append(lv_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        lv_lv_top_mr = np.append(lv_lv_top_mr, [top_rs_df.values], axis=0)
        lv_lv_top_perf += top_perf
    
    # lv+ + lv => lv
    res = sap_sa(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num, 0.1, 0.5, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        lvp_lv_err_prcntl = np.array([err_prcntl_df.values])
        lvp_lv_top_mr = np.array([top_rs_df.values])
        lvp_lv_top_perf = top_perf
    else:
        lvp_lv_err_prcntl = np.append(lvp_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        lvp_lv_top_mr = np.append(lvp_lv_top_mr, [top_rs_df.values], axis=0)
        lvp_lv_top_perf += top_perf
    
    # lmp + voro + lv => lv
    res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, False, \
                 0.15, 0.7, 3)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        l_v_lv_err_prcntl = np.array([err_prcntl_df.values])
        l_v_lv_top_mr = np.array([top_rs_df.values])
        l_v_lv_top_perf = top_perf
    else:
        l_v_lv_err_prcntl = np.append(l_v_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        l_v_lv_top_mr = np.append(l_v_lv_top_mr, [top_rs_df.values], axis=0)
        l_v_lv_top_perf += top_perf
    
    # hstr: lmp + voro + lv => lv
    res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, True, \
                 0, 0.3, 1)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hstr_l_v_lv_err_prcntl = np.array([err_prcntl_df.values])
        hstr_l_v_lv_top_mr = np.array([top_rs_df.values])
        hstr_l_v_lv_top_perf = top_perf
    else:
        hstr_l_v_lv_err_prcntl = np.append(hstr_l_v_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        hstr_l_v_lv_top_mr = np.append(hstr_l_v_lv_top_mr, [top_rs_df.values], axis=0)
        hstr_l_v_lv_top_perf += top_perf
    
    # hstr: lmp + voro => lv
    #res = s_a(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, True)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    #if (rand_seed == rs_start):
    #    hstr_l_v_err_prcntl = np.array([err_prcntl_df.values])
    #    hstr_l_v_top_mr = np.array([top_rs_df.values])
    #    hstr_l_v_top_perf = top_perf
    #else:
    #    hstr_l_v_err_prcntl = np.append(hstr_l_v_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hstr_l_v_top_mr = np.append(hstr_l_v_top_mr, [top_rs_df.values], axis=0)
    #    hstr_l_v_top_perf += top_perf

    # lv- + lv => lv
    #res = sam_sa(lv_shrt_df1, lv_df, lv_conf_colns, perf_coln, smpl_num, 0.2, 0.6, 1)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    lvm_lv_err_prcntl = np.array([err_prcntl_df.values])
    #    lvm_lv_top_mr = np.array([top_rs_df.values])
    #    lvm_lv_top_perf = top_perf
    #else:
    #    lvm_lv_err_prcntl = np.append(lvm_lv_err_prcntl, [err_prcntl_df.values], axis=0)
    #    lvm_lv_top_mr = np.append(lvm_lv_top_mr, [top_rs_df.values], axis=0)
    #    lvm_lv_top_perf += top_perf
        
    # lv-- + lv => lv
    #res = sam_sa(lv_shrt_df2, lv_df, lv_conf_colns, perf_coln, smpl_num, 0.2, 0.6, 1)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    lvmm_lv_err_prcntl = np.array([err_prcntl_df.values])
    #    lvmm_lv_top_mr = np.array([top_rs_df.values])
    #    lvmm_lv_top_perf = top_perf
    #else:
    #    lvmm_lv_err_prcntl = np.append(lvmm_lv_err_prcntl, [err_prcntl_df.values], axis=0)
    #    lvmm_lv_top_mr = np.append(lvmm_lv_top_mr, [top_rs_df.values], axis=0)
    #    lvmm_lv_top_perf += top_perf
    
print "cmbn: lv+ + lv => lv"
cmbn_lvp_err_prcntl_avg = np.sum(cmbn_lvp_err_prcntl, axis=0) / len(cmbn_lvp_err_prcntl)
cmbn_lvp_err_prcntl_df = pd.DataFrame(cmbn_lvp_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_lvp_err_prcntl_df
df2csv(cmbn_lvp_err_prcntl_df, dirname + 'err_prcntl_cmbn_lvp_' + str(smpl_num) + '.csv')
    
print "cmbn: lmp + voro + lv => lv"
cmbn_l_v_err_prcntl_avg = np.sum(cmbn_l_v_err_prcntl, axis=0) / len(cmbn_l_v_err_prcntl)
cmbn_l_v_err_prcntl_df = pd.DataFrame(cmbn_l_v_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_l_v_err_prcntl_df
df2csv(cmbn_l_v_err_prcntl_df, dirname + 'err_prcntl_cmbn_l_v_' + str(smpl_num) + '.csv')

print "lv => lv"
lv_err_prcntl_avg = np.sum(lv_err_prcntl, axis=0) / len(lv_err_prcntl)
lv_err_prcntl_df = pd.DataFrame(lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print lv_err_prcntl_df
df2csv(lv_err_prcntl_df, dirname + 'err_prcntl_lv_' + str(smpl_num) + '.csv')

print "lv + lv => lv"
lv_lv_err_prcntl_avg = np.sum(lv_lv_err_prcntl, axis=0) / len(lv_lv_err_prcntl)
lv_lv_err_prcntl_df = pd.DataFrame(lv_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print lv_lv_err_prcntl_df
df2csv(lv_lv_err_prcntl_df, dirname + 'err_prcntl_lv_lv_' + str(smpl_num) + '.csv')

print "lv+ + lv => lv"
lvp_lv_err_prcntl_avg = np.sum(lvp_lv_err_prcntl, axis=0) / len(lvp_lv_err_prcntl)
lvp_lv_err_prcntl_df = pd.DataFrame(lvp_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print lvp_lv_err_prcntl_df
df2csv(lvp_lv_err_prcntl_df, dirname + 'err_prcntl_lvp_lv_' + str(smpl_num) + '.csv')

print "lmp + voro + lv => lv"
l_v_lv_err_prcntl_avg = np.sum(l_v_lv_err_prcntl, axis=0) / len(l_v_lv_err_prcntl)
l_v_lv_err_prcntl_df = pd.DataFrame(l_v_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print l_v_lv_err_prcntl_df
df2csv(l_v_lv_err_prcntl_df, dirname + 'err_prcntl_l_v_lv_' + str(smpl_num) + '.csv')

print "hstr: lmp + voro + lv => lv"
hstr_l_v_lv_err_prcntl_avg = np.sum(hstr_l_v_lv_err_prcntl, axis=0) / len(hstr_l_v_lv_err_prcntl)
hstr_l_v_lv_err_prcntl_df = pd.DataFrame(hstr_l_v_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hstr_l_v_lv_err_prcntl_df
df2csv(hstr_l_v_lv_err_prcntl_df, dirname + 'err_prcntl_hstr_l_v_lv_' + str(smpl_num) + '.csv')

#print "hstr: lmp + voro => lv"
#hstr_l_v_err_prcntl_avg = np.sum(hstr_l_v_err_prcntl, axis=0) / len(hstr_l_v_err_prcntl)
#hstr_l_v_err_prcntl_df = pd.DataFrame(hstr_l_v_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hstr_l_v_err_prcntl_df
#df2csv(hstr_l_v_err_prcntl_df, dirname + 'err_prcntl_hstr_l_v_' + str(smpl_num) + '.csv')

#print "lv- + lv => lv"
#lvm_lv_err_prcntl_avg = np.sum(lvm_lv_err_prcntl, axis=0) / len(lvm_lv_err_prcntl)
#lvm_lv_err_prcntl_df = pd.DataFrame(lvm_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print lvm_lv_err_prcntl_df
#df2csv(lvm_lv_err_prcntl_df, dirname + 'err_prcntl_lvm_lv_' + str(smpl_num) + '.csv')

#print "lv-- + lv => lv"
#lvmm_lv_err_prcntl_avg = np.sum(lvmm_lv_err_prcntl, axis=0) / len(lvmm_lv_err_prcntl)
#lvmm_lv_err_prcntl_df = pd.DataFrame(lvmm_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print lvmm_lv_err_prcntl_df
#df2csv(lvmm_lv_err_prcntl_df, dirname + 'err_prcntl_lvmm_lv_' + str(smpl_num) + '.csv')

colns = ['cmbn_lvp', 'cmbn_l_v', 'lv', 'lv_lv', \
         'lvp_lv', 'l_v_lv', 'hstr_l_v_lv']
#         'hstr_l_v', 'lvm_lv', 'lvmm_lv']
print "Recall Score (%)"
cmbn_lvp_top_mr_avg = np.sum(cmbn_lvp_top_mr, axis=0) / len(cmbn_lvp_top_mr)
cmbn_l_v_top_mr_avg = np.sum(cmbn_l_v_top_mr, axis=0) / len(cmbn_l_v_top_mr)
lv_top_mr_avg = np.sum(lv_top_mr, axis=0) / len(lv_top_mr)
lv_lv_top_mr_avg = np.sum(lv_lv_top_mr, axis=0) / len(lv_lv_top_mr)
lvp_lv_top_mr_avg = np.sum(lvp_lv_top_mr, axis=0) / len(lvp_lv_top_mr)
l_v_lv_top_mr_avg = np.sum(l_v_lv_top_mr, axis=0) / len(l_v_lv_top_mr)
hstr_l_v_lv_top_mr_avg = np.sum(hstr_l_v_lv_top_mr, axis=0) / len(hstr_l_v_lv_top_mr)
#hstr_l_v_top_mr_avg = np.sum(hstr_l_v_top_mr, axis=0) / len(hstr_l_v_top_mr)
#lvm_lv_top_mr_avg = np.sum(lvm_lv_top_mr, axis=0) / len(lvm_lv_top_mr)
#lvmm_lv_top_mr_avg = np.sum(lvmm_lv_top_mr, axis=0) / len(lvmm_lv_top_mr)
top_mr_avg = np.c_[cmbn_lvp_top_mr_avg, \
                   cmbn_l_v_top_mr_avg[:, -1], \
                   lv_top_mr_avg[:, -1], \
                   lv_lv_top_mr_avg[:, -1], \
                   lvp_lv_top_mr_avg[:, -1], \
                   l_v_lv_top_mr_avg[:, -1], \
                   hstr_l_v_lv_top_mr_avg[:, -1]
#                   hstr_l_v_top_mr_avg[:, -1]
#                   l_v_lv_lv_top_mr_avg[:, -1], \
#                   lvp_lv_top_mr_avg[:, -1], \
#                   lvm_lv_top_mr_avg[:, -1], \
#                   lvmm_lv_top_mr_avg[:, -1]
                  ]
top_rs_df = pd.DataFrame(top_mr_avg, columns=['top_prec', 'top_num']+colns)
print top_rs_df
df2csv(top_rs_df, dirname + 'top_rs_' + str(smpl_num) + '.csv')

print "Top performance [Exection Time (s)]"
rs_num = (rs_end - rs_start) // rs_step
top_perf = np.c_[cmbn_lvp_top_perf, \
                  cmbn_l_v_top_perf, \
                  lv_top_perf, \
                  lv_lv_top_perf, \
                  lvp_lv_top_perf, \
                  l_v_lv_top_perf, \
                  hstr_l_v_lv_top_perf
                 ] / rs_num
top_perf_df = pd.DataFrame(top_perf, columns=colns)
print top_perf_df
df2csv(top_perf_df, dirname + 'top_perf_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'mach_time'
dirname = '../plot/lmp-voro/' + perf_coln + '/'
print gen_top_df(lv_df, perf_coln, 1)

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    lmp_df = lmp_df.sort_values(lmp_conf_colns).reset_index(drop=True)
    voro_df = voro_df.sort_values(voro_conf_colns).reset_index(drop=True)
    lv_df = lv_df.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_in_df = lv_in_df.sort_values(lv_in_conf_colns).reset_index(drop=True)
    lv_shrt_df1 = lv_shrt_df1.sort_values(lv_conf_colns).reset_index(drop=True)
    lv_shrt_df2 = lv_shrt_df2.sort_values(lv_conf_colns).reset_index(drop=True)

    lmp_df = lmp_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    voro_df = voro_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_df = lv_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_in_df = lv_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_shrt_df1 = lv_shrt_df1.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    lv_shrt_df2 = lv_shrt_df2.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # cmbn: lv+ + lv => lv
    res = cmbn_sap(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_lvp_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_lvp_top_mr = np.array([top_rs_df.values])
        cmbn_lvp_top_perf = top_perf
    else:
        cmbn_lvp_err_prcntl = np.append(cmbn_lvp_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_lvp_top_mr = np.append(cmbn_lvp_top_mr, [top_rs_df.values], axis=0)
        cmbn_lvp_top_perf += top_perf
    
    # cmbn: lmp + voro + lv => lv
    res = cmbn_s_a(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_l_v_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_l_v_top_mr = np.array([top_rs_df.values])
        cmbn_l_v_top_perf = top_perf
    else:
        cmbn_l_v_err_prcntl = np.append(cmbn_l_v_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_l_v_top_mr = np.append(cmbn_l_v_top_mr, [top_rs_df.values], axis=0)
        cmbn_l_v_top_perf += top_perf
    
    # drct: lv => lv
    res = drct_sa(lv_df, lv_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        lv_err_prcntl = np.array([err_prcntl_df.values])
        lv_top_mr = np.array([top_rs_df.values])
        lv_top_perf = top_perf
    else:
        lv_err_prcntl = np.append(lv_err_prcntl, [err_prcntl_df.values], axis=0)
        lv_top_mr = np.append(lv_top_mr, [top_rs_df.values], axis=0)
        lv_top_perf += top_perf
    
    # lv + lv => lv
    res = sa_sa(lv_df, lv_conf_colns, perf_coln, smpl_num, 0.4, 4)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        lv_lv_err_prcntl = np.array([err_prcntl_df.values])
        lv_lv_top_mr = np.array([top_rs_df.values])
        lv_lv_top_perf = top_perf
    else:
        lv_lv_err_prcntl = np.append(lv_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        lv_lv_top_mr = np.append(lv_lv_top_mr, [top_rs_df.values], axis=0)
        lv_lv_top_perf += top_perf
    
    # lv+ + lv => lv
    res = sap_sa(lv_in_df, lv_df, [16000, 20000], lv_in_conf_colns, lv_conf_colns, perf_coln, smpl_num, 0.05, 0.85, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        lvp_lv_err_prcntl = np.array([err_prcntl_df.values])
        lvp_lv_top_mr = np.array([top_rs_df.values])
        lvp_lv_top_perf = top_perf
    else:
        lvp_lv_err_prcntl = np.append(lvp_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        lvp_lv_top_mr = np.append(lvp_lv_top_mr, [top_rs_df.values], axis=0)
        lvp_lv_top_perf += top_perf
    
    # lmp + voro + lv => lv
    res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, False, \
                 0.2, 0.55, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        l_v_lv_err_prcntl = np.array([err_prcntl_df.values])
        l_v_lv_top_mr = np.array([top_rs_df.values])
        l_v_lv_top_perf = top_perf
    else:
        l_v_lv_err_prcntl = np.append(l_v_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        l_v_lv_top_mr = np.append(l_v_lv_top_mr, [top_rs_df.values], axis=0)
        l_v_lv_top_perf += top_perf
    
    # hstr: lmp + voro + lv => lv
    res = s_a_sa(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, True, \
                 0, 0.2, 4)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hstr_l_v_lv_err_prcntl = np.array([err_prcntl_df.values])
        hstr_l_v_lv_top_mr = np.array([top_rs_df.values])
        hstr_l_v_lv_top_perf = top_perf
    else:
        hstr_l_v_lv_err_prcntl = np.append(hstr_l_v_lv_err_prcntl, [err_prcntl_df.values], axis=0)
        hstr_l_v_lv_top_mr = np.append(hstr_l_v_lv_top_mr, [top_rs_df.values], axis=0)
        hstr_l_v_lv_top_perf += top_perf
    
    # hstr: lmp + voro => lv
    #res = s_a(lmp_df, voro_df, lv_df, lmp_conf_colns, voro_conf_colns, lv_conf_colns, perf_coln, smpl_num, True)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    #if (rand_seed == rs_start):
    #    hstr_l_v_err_prcntl = np.array([err_prcntl_df.values])
    #    hstr_l_v_top_mr = np.array([top_rs_df.values])
    #    hstr_l_v_top_perf = top_perf
    #else:
    #    hstr_l_v_err_prcntl = np.append(hstr_l_v_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hstr_l_v_top_mr = np.append(hstr_l_v_top_mr, [top_rs_df.values], axis=0)
    #    hstr_l_v_top_perf += top_perf

    # lv- + lv => lv
    #res = sam_sa(lv_shrt_df1, lv_df, lv_conf_colns, perf_coln, smpl_num, 0.2, 0.6, 1)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    lvm_lv_err_prcntl = np.array([err_prcntl_df.values])
    #    lvm_lv_top_mr = np.array([top_rs_df.values])
    #    lvm_lv_top_perf = top_perf
    #else:
    #    lvm_lv_err_prcntl = np.append(lvm_lv_err_prcntl, [err_prcntl_df.values], axis=0)
    #    lvm_lv_top_mr = np.append(lvm_lv_top_mr, [top_rs_df.values], axis=0)
    #    lvm_lv_top_perf += top_perf
        
    # lv-- + lv => lv
    #res = sam_sa(lv_shrt_df2, lv_df, lv_conf_colns, perf_coln, smpl_num, 0.2, 0.6, 1)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    lvmm_lv_err_prcntl = np.array([err_prcntl_df.values])
    #    lvmm_lv_top_mr = np.array([top_rs_df.values])
    #    lvmm_lv_top_perf = top_perf
    #else:
    #    lvmm_lv_err_prcntl = np.append(lvmm_lv_err_prcntl, [err_prcntl_df.values], axis=0)
    #    lvmm_lv_top_mr = np.append(lvmm_lv_top_mr, [top_rs_df.values], axis=0)
    #    lvmm_lv_top_perf += top_perf
    
print "cmbn: lv+ + lv => lv"
cmbn_lvp_err_prcntl_avg = np.sum(cmbn_lvp_err_prcntl, axis=0) / len(cmbn_lvp_err_prcntl)
cmbn_lvp_err_prcntl_df = pd.DataFrame(cmbn_lvp_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_lvp_err_prcntl_df
df2csv(cmbn_lvp_err_prcntl_df, dirname + 'err_prcntl_cmbn_lvp_' + str(smpl_num) + '.csv')
    
print "cmbn: lmp + voro + lv => lv"
cmbn_l_v_err_prcntl_avg = np.sum(cmbn_l_v_err_prcntl, axis=0) / len(cmbn_l_v_err_prcntl)
cmbn_l_v_err_prcntl_df = pd.DataFrame(cmbn_l_v_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_l_v_err_prcntl_df
df2csv(cmbn_l_v_err_prcntl_df, dirname + 'err_prcntl_cmbn_l_v_' + str(smpl_num) + '.csv')

print "lv => lv"
lv_err_prcntl_avg = np.sum(lv_err_prcntl, axis=0) / len(lv_err_prcntl)
lv_err_prcntl_df = pd.DataFrame(lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print lv_err_prcntl_df
df2csv(lv_err_prcntl_df, dirname + 'err_prcntl_lv_' + str(smpl_num) + '.csv')

print "lv + lv => lv"
lv_lv_err_prcntl_avg = np.sum(lv_lv_err_prcntl, axis=0) / len(lv_lv_err_prcntl)
lv_lv_err_prcntl_df = pd.DataFrame(lv_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print lv_lv_err_prcntl_df
df2csv(lv_lv_err_prcntl_df, dirname + 'err_prcntl_lv_lv_' + str(smpl_num) + '.csv')

print "lv+ + lv => lv"
lvp_lv_err_prcntl_avg = np.sum(lvp_lv_err_prcntl, axis=0) / len(lvp_lv_err_prcntl)
lvp_lv_err_prcntl_df = pd.DataFrame(lvp_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print lvp_lv_err_prcntl_df
df2csv(lvp_lv_err_prcntl_df, dirname + 'err_prcntl_lvp_lv_' + str(smpl_num) + '.csv')

print "lmp + voro + lv => lv"
l_v_lv_err_prcntl_avg = np.sum(l_v_lv_err_prcntl, axis=0) / len(l_v_lv_err_prcntl)
l_v_lv_err_prcntl_df = pd.DataFrame(l_v_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print l_v_lv_err_prcntl_df
df2csv(l_v_lv_err_prcntl_df, dirname + 'err_prcntl_l_v_lv_' + str(smpl_num) + '.csv')

print "hstr: lmp + voro + lv => lv"
hstr_l_v_lv_err_prcntl_avg = np.sum(hstr_l_v_lv_err_prcntl, axis=0) / len(hstr_l_v_lv_err_prcntl)
hstr_l_v_lv_err_prcntl_df = pd.DataFrame(hstr_l_v_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hstr_l_v_lv_err_prcntl_df
df2csv(hstr_l_v_lv_err_prcntl_df, dirname + 'err_prcntl_hstr_l_v_lv_' + str(smpl_num) + '.csv')

#print "hstr: lmp + voro => lv"
#hstr_l_v_err_prcntl_avg = np.sum(hstr_l_v_err_prcntl, axis=0) / len(hstr_l_v_err_prcntl)
#hstr_l_v_err_prcntl_df = pd.DataFrame(hstr_l_v_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hstr_l_v_err_prcntl_df
#df2csv(hstr_l_v_err_prcntl_df, dirname + 'err_prcntl_hstr_l_v_' + str(smpl_num) + '.csv')

#print "lv- + lv => lv"
#lvm_lv_err_prcntl_avg = np.sum(lvm_lv_err_prcntl, axis=0) / len(lvm_lv_err_prcntl)
#lvm_lv_err_prcntl_df = pd.DataFrame(lvm_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print lvm_lv_err_prcntl_df
#df2csv(lvm_lv_err_prcntl_df, dirname + 'err_prcntl_lvm_lv_' + str(smpl_num) + '.csv')

#print "lv-- + lv => lv"
#lvmm_lv_err_prcntl_avg = np.sum(lvmm_lv_err_prcntl, axis=0) / len(lvmm_lv_err_prcntl)
#lvmm_lv_err_prcntl_df = pd.DataFrame(lvmm_lv_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print lvmm_lv_err_prcntl_df
#df2csv(lvmm_lv_err_prcntl_df, dirname + 'err_prcntl_lvmm_lv_' + str(smpl_num) + '.csv')

colns = ['cmbn_lvp', 'cmbn_l_v', 'lv', 'lv_lv', \
         'lvp_lv', 'l_v_lv', 'hstr_l_v_lv']
#         'hstr_l_v', 'lvm_lv', 'lvmm_lv']
print "Recall Score (%)"
cmbn_lvp_top_mr_avg = np.sum(cmbn_lvp_top_mr, axis=0) / len(cmbn_lvp_top_mr)
cmbn_l_v_top_mr_avg = np.sum(cmbn_l_v_top_mr, axis=0) / len(cmbn_l_v_top_mr)
lv_top_mr_avg = np.sum(lv_top_mr, axis=0) / len(lv_top_mr)
lv_lv_top_mr_avg = np.sum(lv_lv_top_mr, axis=0) / len(lv_lv_top_mr)
lvp_lv_top_mr_avg = np.sum(lvp_lv_top_mr, axis=0) / len(lvp_lv_top_mr)
l_v_lv_top_mr_avg = np.sum(l_v_lv_top_mr, axis=0) / len(l_v_lv_top_mr)
hstr_l_v_lv_top_mr_avg = np.sum(hstr_l_v_lv_top_mr, axis=0) / len(hstr_l_v_lv_top_mr)
#hstr_l_v_top_mr_avg = np.sum(hstr_l_v_top_mr, axis=0) / len(hstr_l_v_top_mr)
#lvm_lv_top_mr_avg = np.sum(lvm_lv_top_mr, axis=0) / len(lvm_lv_top_mr)
#lvmm_lv_top_mr_avg = np.sum(lvmm_lv_top_mr, axis=0) / len(lvmm_lv_top_mr)
top_mr_avg = np.c_[cmbn_lvp_top_mr_avg, \
                   cmbn_l_v_top_mr_avg[:, -1], \
                   lv_top_mr_avg[:, -1], \
                   lv_lv_top_mr_avg[:, -1], \
                   lvp_lv_top_mr_avg[:, -1], \
                   l_v_lv_top_mr_avg[:, -1], \
                   hstr_l_v_lv_top_mr_avg[:, -1]
#                   hstr_l_v_top_mr_avg[:, -1]
#                   l_v_lv_lv_top_mr_avg[:, -1], \
#                   lvp_lv_top_mr_avg[:, -1], \
#                   lvm_lv_top_mr_avg[:, -1], \
#                   lvmm_lv_top_mr_avg[:, -1]
                  ]
top_rs_df = pd.DataFrame(top_mr_avg, columns=['top_prec', 'top_num']+colns)
print top_rs_df
df2csv(top_rs_df, dirname + 'top_rs_' + str(smpl_num) + '.csv')

print "Top performance [Machine time (cores * hours)]"
rs_num = (rs_end - rs_start) // rs_step
top_perf = np.c_[cmbn_lvp_top_perf, \
                  cmbn_l_v_top_perf, \
                  lv_top_perf, \
                  lv_lv_top_perf, \
                  lvp_lv_top_perf, \
                  l_v_lv_top_perf, \
                  hstr_l_v_lv_top_perf
                 ] / rs_num
top_perf_df = pd.DataFrame(top_perf, columns=colns)
print top_perf_df
df2csv(top_perf_df, dirname + 'top_perf_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'run_time'
dirname = '../plot/ht-sw/' + perf_coln + '/'
print gen_top_df(hs_df, perf_coln, 1)

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    ht_df = ht_df.sort_values(ht_conf_colns).reset_index(drop=True)
    sw_df = sw_df.sort_values(sw_conf_colns).reset_index(drop=True)
    hs_df = hs_df.sort_values(hs_conf_colns).reset_index(drop=True)
    hs_in_df = hs_in_df.sort_values(hs_in_conf_colns).reset_index(drop=True)
    #hs_shrt_df1 = hs_shrt_df1.sort_values(hs_conf_colns).reset_index(drop=True)
    #hs_shrt_df2 = hs_shrt_df2.sort_values(hs_conf_colns).reset_index(drop=True)

    ht_df = ht_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    sw_df = sw_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_df = hs_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_in_df = hs_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    #hs_shrt_df1 = hs_shrt_df1.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    #hs_shrt_df2 = hs_shrt_df2.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # cmbn: hs+ + hs => hs
    res = cmbn_sap(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_hsp_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_hsp_top_mr = np.array([top_rs_df.values])
        cmbn_hsp_top_perf = top_perf
    else:
        cmbn_hsp_err_prcntl = np.append(cmbn_hsp_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_hsp_top_mr = np.append(cmbn_hsp_top_mr, [top_rs_df.values], axis=0)
        cmbn_hsp_top_perf += top_perf
    
    # cmbn: ht + sw + hs => hs
    res = cmbn_s_a(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_h_s_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_h_s_top_mr = np.array([top_rs_df.values])
        cmbn_h_s_top_perf = top_perf
    else:
        cmbn_h_s_err_prcntl = np.append(cmbn_h_s_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_h_s_top_mr = np.append(cmbn_h_s_top_mr, [top_rs_df.values], axis=0)
        cmbn_h_s_top_perf += top_perf
    
    # drct: hs => hs
    res = drct_sa(hs_df, hs_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        hs_err_prcntl = np.array([err_prcntl_df.values])
        hs_top_mr = np.array([top_rs_df.values])
        hs_top_perf = top_perf
    else:
        hs_err_prcntl = np.append(hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hs_top_mr = np.append(hs_top_mr, [top_rs_df.values], axis=0)
        hs_top_perf += top_perf
    
    # hs + hs => hs
    res = sa_sa(hs_df, hs_conf_colns, perf_coln, smpl_num, 0.4, 5)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hs_hs_err_prcntl = np.array([err_prcntl_df.values])
        hs_hs_top_mr = np.array([top_rs_df.values])
        hs_hs_top_perf = top_perf
    else:
        hs_hs_err_prcntl = np.append(hs_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hs_hs_top_mr = np.append(hs_hs_top_mr, [top_rs_df.values], axis=0)
        hs_hs_top_perf += top_perf
    
    # hs+ + hs => hs
    res = sap_sa(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                 0.05, 0.8, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hsp_hs_err_prcntl = np.array([err_prcntl_df.values])
        hsp_hs_top_mr = np.array([top_rs_df.values])
        hsp_hs_top_perf = top_perf
    else:
        hsp_hs_err_prcntl = np.append(hsp_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hsp_hs_top_mr = np.append(hsp_hs_top_mr, [top_rs_df.values], axis=0)
        hsp_hs_top_perf += top_perf
    
    # ht + sw + hs => hs
    res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, False, \
                 0.15, 0.3, 1)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        h_s_hs_err_prcntl = np.array([err_prcntl_df.values])
        h_s_hs_top_mr = np.array([top_rs_df.values])
        h_s_hs_top_perf = top_perf
    else:
        h_s_hs_err_prcntl = np.append(h_s_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        h_s_hs_top_mr = np.append(h_s_hs_top_mr, [top_rs_df.values], axis=0)
        h_s_hs_top_perf += top_perf
    
    # hstr: ht + sw + hs => hs
    res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, True, \
                 0, 0.3, 1)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hstr_h_s_hs_err_prcntl = np.array([err_prcntl_df.values])
        hstr_h_s_hs_top_mr = np.array([top_rs_df.values])
        hstr_h_s_hs_top_perf = top_perf
    else:
        hstr_h_s_hs_err_prcntl = np.append(hstr_h_s_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hstr_h_s_hs_top_mr = np.append(hstr_h_s_hs_top_mr, [top_rs_df.values], axis=0)
        hstr_h_s_hs_top_perf += top_perf
    
    # hstr: ht + sw => hs
    #res = s_a(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, True)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    #if (rand_seed == rs_start):
    #    hstr_h_s_err_prcntl = np.array([err_prcntl_df.values])
    #    hstr_h_s_top_mr = np.array([top_rs_df.values])
    #    hstr_h_s_top_perf = top_perf
    #else:
    #    hstr_h_s_err_prcntl = np.append(hstr_h_s_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hstr_h_s_top_mr = np.append(hstr_h_s_top_mr, [top_rs_df.values], axis=0)
    #    hstr_h_s_top_perf += top_perf
    
    # hs- + hs => hs
    #res = sam_sa(hs_shrt_df1, hs_df, hs_conf_colns, perf_coln, smpl_num)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    hsm_hs_err_prcntl = np.array([err_prcntl_df.values])
    #    hsm_hs_top_mr = np.array([top_rs_df.values])
    #    hsm_hs_top_perf = top_perf
    #else:
    #    hsm_hs_err_prcntl = np.append(hsm_hs_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hsm_hs_top_mr = np.append(hsm_hs_top_mr, [top_rs_df.values], axis=0)
    #    hsm_hs_top_perf += top_perf
    
    # hs-- + hs => hs
    #res = sam_sa(hs_shrt_df2, hs_df, hs_conf_colns, perf_coln, smpl_num)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    hsmm_hs_err_prcntl = np.array([err_prcntl_df.values])
    #    hsmm_hs_top_mr = np.array([top_rs_df.values])
    #    hsmm_hs_top_perf = top_perf
    #else:
    #    hsmm_hs_err_prcntl = np.append(hsmm_hs_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hsmm_hs_top_mr = np.append(hsmm_hs_top_mr, [top_rs_df.values], axis=0)
    #    hsmm_hs_top_perf += top_perf

print "cmbn: hs+ + hs => hs"
cmbn_hsp_err_prcntl_avg = np.sum(cmbn_hsp_err_prcntl, axis=0) / len(cmbn_hsp_err_prcntl)
cmbn_hsp_err_prcntl_df = pd.DataFrame(cmbn_hsp_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_hsp_err_prcntl_df
df2csv(cmbn_hsp_err_prcntl_df, dirname + 'err_prcntl_cmbn_hsp_' + str(smpl_num) + '.csv')

print "cmbn: ht + sw + hs => hs"
cmbn_h_s_err_prcntl_avg = np.sum(cmbn_h_s_err_prcntl, axis=0) / len(cmbn_h_s_err_prcntl)
cmbn_h_s_err_prcntl_df = pd.DataFrame(cmbn_h_s_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_h_s_err_prcntl_df
df2csv(cmbn_h_s_err_prcntl_df, dirname + 'err_prcntl_cmbn_h_s_' + str(smpl_num) + '.csv')

print "hs => hs"
hs_err_prcntl_avg = np.sum(hs_err_prcntl, axis=0) / len(hs_err_prcntl)
hs_err_prcntl_df = pd.DataFrame(hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hs_err_prcntl_df
df2csv(hs_err_prcntl_df, dirname + 'err_prcntl_hs_' + str(smpl_num) + '.csv')

print "hs + hs => hs"
hs_hs_err_prcntl_avg = np.sum(hs_hs_err_prcntl, axis=0) / len(hs_hs_err_prcntl)
hs_hs_err_prcntl_df = pd.DataFrame(hs_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hs_hs_err_prcntl_df
df2csv(hs_hs_err_prcntl_df, dirname + 'err_prcntl_hs_hs_' + str(smpl_num) + '.csv')

print "hs+ + hs + hs => hs"
hsp_hs_err_prcntl_avg = np.sum(hsp_hs_err_prcntl, axis=0) / len(hsp_hs_err_prcntl)
hsp_hs_err_prcntl_df = pd.DataFrame(hsp_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hsp_hs_err_prcntl_df
df2csv(hsp_hs_err_prcntl_df, dirname + 'err_prcntl_hsp_hs_' + str(smpl_num) + '.csv')

print "ht + sw + hs => hs"
h_s_hs_err_prcntl_avg = np.sum(h_s_hs_err_prcntl, axis=0) / len(h_s_hs_err_prcntl)
h_s_hs_err_prcntl_df = pd.DataFrame(h_s_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print h_s_hs_err_prcntl_df
df2csv(h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_s_hs_' + str(smpl_num) + '.csv')

print "hstr: ht + sw + hs => hs"
hstr_h_s_hs_err_prcntl_avg = np.sum(hstr_h_s_hs_err_prcntl, axis=0) / len(hstr_h_s_hs_err_prcntl)
hstr_h_s_hs_err_prcntl_df = pd.DataFrame(hstr_h_s_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hstr_h_s_hs_err_prcntl_df
df2csv(hstr_h_s_hs_err_prcntl_df, dirname + 'err_prcntl_hstr_h_s_hs_' + str(smpl_num) + '.csv')

#print "hstr: ht + sw => hs"
#hstr_h_s_err_prcntl_avg = np.sum(hstr_h_s_err_prcntl, axis=0) / len(hstr_h_s_err_prcntl)
#hstr_h_s_err_prcntl_df = pd.DataFrame(hstr_h_s_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hstr_h_s_err_prcntl_df
#df2csv(hstr_h_s_err_prcntl_df, dirname + 'err_prcntl_hstr_h_s_' + str(smpl_num) + '.csv')

#print "hs- + hs => hs"
#hsm_hs_err_prcntl_avg = np.sum(hsm_hs_err_prcntl, axis=0) / len(hsm_hs_err_prcntl)
#hsm_hs_err_prcntl_df = pd.DataFrame(hsm_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hsm_hs_err_prcntl_df
#df2csv(hsm_hs_err_prcntl_df, dirname + 'err_prcntl_hsm_hs_' + str(smpl_num) + '.csv')

#print "hs-- + hs => hs"
#hsmm_hs_err_prcntl_avg = np.sum(hsmm_hs_err_prcntl, axis=0) / len(hsmm_hs_err_prcntl)
#hsmm_hs_err_prcntl_df = pd.DataFrame(hsmm_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hsmm_hs_err_prcntl_df
#df2csv(hsmm_hs_err_prcntl_df, dirname + 'err_prcntl_hsmm_hs_' + str(smpl_num) + '.csv')

colns = ['cmbn_hsp', 'cmbn_h_s', 'hs', 'hs_hs', \
         'hsp_hs', 'h_s_hs', 'hstr_h_s_hs']
#         'hstr_h_s', 'hsm_hs', 'hsmm_hs']

print "Recall score (%)"
cmbn_hsp_top_mr_avg = np.sum(cmbn_hsp_top_mr, axis=0) / len(cmbn_hsp_top_mr)
cmbn_h_s_top_mr_avg = np.sum(cmbn_h_s_top_mr, axis=0) / len(cmbn_h_s_top_mr)
hs_top_mr_avg = np.sum(hs_top_mr, axis=0) / len(hs_top_mr)
hs_hs_top_mr_avg = np.sum(hs_hs_top_mr, axis=0) / len(hs_hs_top_mr)
hsp_hs_top_mr_avg = np.sum(hsp_hs_top_mr, axis=0) / len(hsp_hs_top_mr)
h_s_hs_top_mr_avg = np.sum(h_s_hs_top_mr, axis=0) / len(h_s_hs_top_mr)
hstr_h_s_hs_top_mr_avg = np.sum(hstr_h_s_hs_top_mr, axis=0) / len(hstr_h_s_hs_top_mr)
#hstr_h_s_top_mr_avg = np.sum(hstr_h_s_top_mr, axis=0) / len(hstr_h_s_top_mr)
#hsm_hs_top_mr_avg = np.sum(hsm_hs_top_mr, axis=0) / len(hsm_hs_top_mr)
#hsmm_hs_top_mr_avg = np.sum(hsmm_hs_top_mr, axis=0) / len(hsmm_hs_top_mr)
top_mr_avg = np.c_[cmbn_hsp_top_mr_avg, \
                   cmbn_h_s_top_mr_avg[:, -1], \
                   hs_top_mr_avg[:, -1], \
                   hs_hs_top_mr_avg[:, -1], \
                   hsp_hs_top_mr_avg[:, -1], \
                   h_s_hs_top_mr_avg[:, -1], \
                   hstr_h_s_hs_top_mr_avg[:, -1]
#                   hstr_h_s_top_mr_avg[:, -1], \
#                   hsm_hs_top_mr_avg[:, -1], \
#                   hsmm_hs_top_mr_avg[:, -1]
                  ]
top_rs_df = pd.DataFrame(top_mr_avg, columns=['top_prec', 'top_num']+colns)
print top_rs_df
df2csv(top_rs_df, dirname + 'top_rs_' + str(smpl_num) + '.csv')

print "Top performance [Exection Time (s)]"
rs_num = (rs_end - rs_start) // rs_step
top_perf = np.c_[cmbn_hsp_top_perf, \
                  cmbn_h_s_top_perf, \
                  hs_top_perf, \
                  hs_hs_top_perf, \
                  hsp_hs_top_perf, \
                  h_s_hs_top_perf, \
                  hstr_h_s_hs_top_perf
#                  hstr_h_s_top_perf, \
#                  hsm_hs_top_perf, \
#                  hsmm_hs_top_perf
                 ] / rs_num
top_perf_df = pd.DataFrame(top_perf, columns=colns)
print top_perf_df
df2csv(top_perf_df, dirname + 'top_perf_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
import time
startT = time.time()

smpl_num = 100

perf_coln = 'mach_time'
dirname = '../plot/ht-sw/' + perf_coln + '/'
print gen_top_df(hs_df, perf_coln, 1)

rs_start = 1
rs_end = 101
rs_step = 1
for rand_seed in range(rs_start, rs_end, rs_step):
    ht_df = ht_df.sort_values(ht_conf_colns).reset_index(drop=True)
    sw_df = sw_df.sort_values(sw_conf_colns).reset_index(drop=True)
    hs_df = hs_df.sort_values(hs_conf_colns).reset_index(drop=True)
    hs_in_df = hs_in_df.sort_values(hs_in_conf_colns).reset_index(drop=True)
    #hs_shrt_df1 = hs_shrt_df1.sort_values(hs_conf_colns).reset_index(drop=True)
    #hs_shrt_df2 = hs_shrt_df2.sort_values(hs_conf_colns).reset_index(drop=True)

    ht_df = ht_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    sw_df = sw_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_df = hs_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    hs_in_df = hs_in_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    #hs_shrt_df1 = hs_shrt_df1.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    #hs_shrt_df2 = hs_shrt_df2.sample(frac=1, random_state=rand_seed).reset_index(drop=True)
    
    # cmbn: hs+ + hs => hs
    res = cmbn_sap(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_hsp_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_hsp_top_mr = np.array([top_rs_df.values])
        cmbn_hsp_top_perf = top_perf
    else:
        cmbn_hsp_err_prcntl = np.append(cmbn_hsp_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_hsp_top_mr = np.append(cmbn_hsp_top_mr, [top_rs_df.values], axis=0)
        cmbn_hsp_top_perf += top_perf
    
    # cmbn: ht + sw + hs => hs
    res = cmbn_s_a(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        cmbn_h_s_err_prcntl = np.array([err_prcntl_df.values])
        cmbn_h_s_top_mr = np.array([top_rs_df.values])
        cmbn_h_s_top_perf = top_perf
    else:
        cmbn_h_s_err_prcntl = np.append(cmbn_h_s_err_prcntl, [err_prcntl_df.values], axis=0)
        cmbn_h_s_top_mr = np.append(cmbn_h_s_top_mr, [top_rs_df.values], axis=0)
        cmbn_h_s_top_perf += top_perf
    
    # drct: hs => hs
    res = drct_sa(hs_df, hs_conf_colns, perf_coln, smpl_num)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    if (rand_seed == rs_start):
        hs_err_prcntl = np.array([err_prcntl_df.values])
        hs_top_mr = np.array([top_rs_df.values])
        hs_top_perf = top_perf
    else:
        hs_err_prcntl = np.append(hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hs_top_mr = np.append(hs_top_mr, [top_rs_df.values], axis=0)
        hs_top_perf += top_perf
    
    # hs + hs => hs
    res = sa_sa(hs_df, hs_conf_colns, perf_coln, smpl_num, 0.5, 4)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hs_hs_err_prcntl = np.array([err_prcntl_df.values])
        hs_hs_top_mr = np.array([top_rs_df.values])
        hs_hs_top_perf = top_perf
    else:
        hs_hs_err_prcntl = np.append(hs_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hs_hs_top_mr = np.append(hs_hs_top_mr, [top_rs_df.values], axis=0)
        hs_hs_top_perf += top_perf
    
    # hs+ + hs => hs
    res = sap_sa(hs_in_df, hs_df, [2048, 2048, 1024], hs_in_conf_colns, hs_conf_colns, perf_coln, smpl_num, \
                 0.2, 0.7, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hsp_hs_err_prcntl = np.array([err_prcntl_df.values])
        hsp_hs_top_mr = np.array([top_rs_df.values])
        hsp_hs_top_perf = top_perf
    else:
        hsp_hs_err_prcntl = np.append(hsp_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hsp_hs_top_mr = np.append(hsp_hs_top_mr, [top_rs_df.values], axis=0)
        hsp_hs_top_perf += top_perf
    
    # ht + sw + hs => hs
    res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, False, \
                 0.2, 0.35, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        h_s_hs_err_prcntl = np.array([err_prcntl_df.values])
        h_s_hs_top_mr = np.array([top_rs_df.values])
        h_s_hs_top_perf = top_perf
    else:
        h_s_hs_err_prcntl = np.append(h_s_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        h_s_hs_top_mr = np.append(h_s_hs_top_mr, [top_rs_df.values], axis=0)
        h_s_hs_top_perf += top_perf
    
    # hstr: ht + sw + hs => hs
    res = s_a_sa(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, True, \
                 0, 0.6, 2)
    top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    if (rand_seed == rs_start):
        hstr_h_s_hs_err_prcntl = np.array([err_prcntl_df.values])
        hstr_h_s_hs_top_mr = np.array([top_rs_df.values])
        hstr_h_s_hs_top_perf = top_perf
    else:
        hstr_h_s_hs_err_prcntl = np.append(hstr_h_s_hs_err_prcntl, [err_prcntl_df.values], axis=0)
        hstr_h_s_hs_top_mr = np.append(hstr_h_s_hs_top_mr, [top_rs_df.values], axis=0)
        hstr_h_s_hs_top_perf += top_perf
    
    # hstr: ht + sw => hs
    #res = s_a(ht_df, sw_df, hs_df, ht_conf_colns, sw_conf_colns, hs_conf_colns, perf_coln, smpl_num, True)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[3]
    #if (rand_seed == rs_start):
    #    hstr_h_s_err_prcntl = np.array([err_prcntl_df.values])
    #    hstr_h_s_top_mr = np.array([top_rs_df.values])
    #    hstr_h_s_top_perf = top_perf
    #else:
    #    hstr_h_s_err_prcntl = np.append(hstr_h_s_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hstr_h_s_top_mr = np.append(hstr_h_s_top_mr, [top_rs_df.values], axis=0)
    #    hstr_h_s_top_perf += top_perf
    
    # hs- + hs => hs
    #res = sam_sa(hs_shrt_df1, hs_df, hs_conf_colns, perf_coln, smpl_num)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    hsm_hs_err_prcntl = np.array([err_prcntl_df.values])
    #    hsm_hs_top_mr = np.array([top_rs_df.values])
    #    hsm_hs_top_perf = top_perf
    #else:
    #    hsm_hs_err_prcntl = np.append(hsm_hs_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hsm_hs_top_mr = np.append(hsm_hs_top_mr, [top_rs_df.values], axis=0)
    #    hsm_hs_top_perf += top_perf
    
    # hs-- + hs => hs
    #res = sam_sa(hs_shrt_df2, hs_df, hs_conf_colns, perf_coln, smpl_num)
    #top_rs_df, err_prcntl_df, top_perf = res[0], res[1], res[5]
    #if (rand_seed == rs_start):
    #    hsmm_hs_err_prcntl = np.array([err_prcntl_df.values])
    #    hsmm_hs_top_mr = np.array([top_rs_df.values])
    #    hsmm_hs_top_perf = top_perf
    #else:
    #    hsmm_hs_err_prcntl = np.append(hsmm_hs_err_prcntl, [err_prcntl_df.values], axis=0)
    #    hsmm_hs_top_mr = np.append(hsmm_hs_top_mr, [top_rs_df.values], axis=0)
    #    hsmm_hs_top_perf += top_perf

print "cmbn: hs+ + hs => hs"
cmbn_hsp_err_prcntl_avg = np.sum(cmbn_hsp_err_prcntl, axis=0) / len(cmbn_hsp_err_prcntl)
cmbn_hsp_err_prcntl_df = pd.DataFrame(cmbn_hsp_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_hsp_err_prcntl_df
df2csv(cmbn_hsp_err_prcntl_df, dirname + 'err_prcntl_cmbn_hsp_' + str(smpl_num) + '.csv')

print "cmbn: ht + sw + hs => hs"
cmbn_h_s_err_prcntl_avg = np.sum(cmbn_h_s_err_prcntl, axis=0) / len(cmbn_h_s_err_prcntl)
cmbn_h_s_err_prcntl_df = pd.DataFrame(cmbn_h_s_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print cmbn_h_s_err_prcntl_df
df2csv(cmbn_h_s_err_prcntl_df, dirname + 'err_prcntl_cmbn_h_s_' + str(smpl_num) + '.csv')

print "hs => hs"
hs_err_prcntl_avg = np.sum(hs_err_prcntl, axis=0) / len(hs_err_prcntl)
hs_err_prcntl_df = pd.DataFrame(hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hs_err_prcntl_df
df2csv(hs_err_prcntl_df, dirname + 'err_prcntl_hs_' + str(smpl_num) + '.csv')

print "hs + hs => hs"
hs_hs_err_prcntl_avg = np.sum(hs_hs_err_prcntl, axis=0) / len(hs_hs_err_prcntl)
hs_hs_err_prcntl_df = pd.DataFrame(hs_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hs_hs_err_prcntl_df
df2csv(hs_hs_err_prcntl_df, dirname + 'err_prcntl_hs_hs_' + str(smpl_num) + '.csv')

print "hs+ + hs + hs => hs"
hsp_hs_err_prcntl_avg = np.sum(hsp_hs_err_prcntl, axis=0) / len(hsp_hs_err_prcntl)
hsp_hs_err_prcntl_df = pd.DataFrame(hsp_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hsp_hs_err_prcntl_df
df2csv(hsp_hs_err_prcntl_df, dirname + 'err_prcntl_hsp_hs_' + str(smpl_num) + '.csv')

print "ht + sw + hs => hs"
h_s_hs_err_prcntl_avg = np.sum(h_s_hs_err_prcntl, axis=0) / len(h_s_hs_err_prcntl)
h_s_hs_err_prcntl_df = pd.DataFrame(h_s_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print h_s_hs_err_prcntl_df
df2csv(h_s_hs_err_prcntl_df, dirname + 'err_prcntl_h_s_hs_' + str(smpl_num) + '.csv')

print "hstr: ht + sw + hs => hs"
hstr_h_s_hs_err_prcntl_avg = np.sum(hstr_h_s_hs_err_prcntl, axis=0) / len(hstr_h_s_hs_err_prcntl)
hstr_h_s_hs_err_prcntl_df = pd.DataFrame(hstr_h_s_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
print hstr_h_s_hs_err_prcntl_df
df2csv(hstr_h_s_hs_err_prcntl_df, dirname + 'err_prcntl_hstr_h_s_hs_' + str(smpl_num) + '.csv')

#print "hstr: ht + sw => hs"
#hstr_h_s_err_prcntl_avg = np.sum(hstr_h_s_err_prcntl, axis=0) / len(hstr_h_s_err_prcntl)
#hstr_h_s_err_prcntl_df = pd.DataFrame(hstr_h_s_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hstr_h_s_err_prcntl_df
#df2csv(hstr_h_s_err_prcntl_df, dirname + 'err_prcntl_hstr_h_s_' + str(smpl_num) + '.csv')

#print "hs- + hs => hs"
#hsm_hs_err_prcntl_avg = np.sum(hsm_hs_err_prcntl, axis=0) / len(hsm_hs_err_prcntl)
#hsm_hs_err_prcntl_df = pd.DataFrame(hsm_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hsm_hs_err_prcntl_df
#df2csv(hsm_hs_err_prcntl_df, dirname + 'err_prcntl_hsm_hs_' + str(smpl_num) + '.csv')

#print "hs-- + hs => hs"
#hsmm_hs_err_prcntl_avg = np.sum(hsmm_hs_err_prcntl, axis=0) / len(hsmm_hs_err_prcntl)
#hsmm_hs_err_prcntl_df = pd.DataFrame(hsmm_hs_err_prcntl_avg, columns=['top_perc'] + range(0, 101, 10))
#print hsmm_hs_err_prcntl_df
#df2csv(hsmm_hs_err_prcntl_df, dirname + 'err_prcntl_hsmm_hs_' + str(smpl_num) + '.csv')

colns = ['cmbn_hsp', 'cmbn_h_s', 'hs', 'hs_hs', \
         'hsp_hs', 'h_s_hs', 'hstr_h_s_hs']
#         'hstr_h_s', 'hsm_hs', 'hsmm_hs']

print "Recall score (%)"
cmbn_hsp_top_mr_avg = np.sum(cmbn_hsp_top_mr, axis=0) / len(cmbn_hsp_top_mr)
cmbn_h_s_top_mr_avg = np.sum(cmbn_h_s_top_mr, axis=0) / len(cmbn_h_s_top_mr)
hs_top_mr_avg = np.sum(hs_top_mr, axis=0) / len(hs_top_mr)
hs_hs_top_mr_avg = np.sum(hs_hs_top_mr, axis=0) / len(hs_hs_top_mr)
hsp_hs_top_mr_avg = np.sum(hsp_hs_top_mr, axis=0) / len(hsp_hs_top_mr)
h_s_hs_top_mr_avg = np.sum(h_s_hs_top_mr, axis=0) / len(h_s_hs_top_mr)
hstr_h_s_hs_top_mr_avg = np.sum(hstr_h_s_hs_top_mr, axis=0) / len(hstr_h_s_hs_top_mr)
#hstr_h_s_top_mr_avg = np.sum(hstr_h_s_top_mr, axis=0) / len(hstr_h_s_top_mr)
#hsm_hs_top_mr_avg = np.sum(hsm_hs_top_mr, axis=0) / len(hsm_hs_top_mr)
#hsmm_hs_top_mr_avg = np.sum(hsmm_hs_top_mr, axis=0) / len(hsmm_hs_top_mr)
top_mr_avg = np.c_[cmbn_hsp_top_mr_avg, \
                   cmbn_h_s_top_mr_avg[:, -1], \
                   hs_top_mr_avg[:, -1], \
                   hs_hs_top_mr_avg[:, -1], \
                   hsp_hs_top_mr_avg[:, -1], \
                   h_s_hs_top_mr_avg[:, -1], \
                   hstr_h_s_hs_top_mr_avg[:, -1]
#                   hstr_h_s_top_mr_avg[:, -1], \
#                   hsm_hs_top_mr_avg[:, -1], \
#                   hsmm_hs_top_mr_avg[:, -1]
                  ]
top_rs_df = pd.DataFrame(top_mr_avg, columns=['top_prec', 'top_num']+colns)
print top_rs_df
df2csv(top_rs_df, dirname + 'top_rs_' + str(smpl_num) + '.csv')

print "Top performance [Machine time (cores * hours)]"
rs_num = (rs_end - rs_start) // rs_step
top_perf = np.c_[cmbn_hsp_top_perf, \
                  cmbn_h_s_top_perf, \
                  hs_top_perf, \
                  hs_hs_top_perf, \
                  hsp_hs_top_perf, \
                  h_s_hs_top_perf, \
                  hstr_h_s_hs_top_perf
#                  hstr_h_s_top_perf, \
#                  hsm_hs_top_perf, \
#                  hsmm_hs_top_perf
                 ] / rs_num
top_perf_df = pd.DataFrame(top_perf, columns=colns)
print top_perf_df
df2csv(top_perf_df, dirname + 'top_perf_' + str(smpl_num) + '.csv')

runtime = time.time() - startT
print "Execution time is %0.4f s." % runtime

In [ ]:
smpl_num = 100

print "lmp => lmp"
pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(lmp_df.head(smpl_num), lmp_df, lmp_conf_colns, perf_coln, \
                                                            lmp_df.shape[0] * 0.005)
find_top_rank(pred_top_smpl, lmp_df, lmp_conf_colns, perf_coln)
lmp_timeout = max(get_vld_df(lmp_df)['run_time'].values) * 1.1
machTime = app_mach_time(lmp_df.head(smpl_num), lmp_timeout)
print "Machine time = %5.1f" % machTime

print "voro => voro"
pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(voro_df.head(smpl_num), voro_df, \
                                                            voro_conf_colns, perf_coln, \
                                                            voro_df.shape[0] * 0.005)
find_top_rank(pred_top_smpl, voro_df, voro_conf_colns, perf_coln)
voro_timeout = max(get_vld_df(voro_df)['run_time'].values) * 1.1
machTime = app_mach_time(voro_df.head(smpl_num), voro_timeout)
print "Machine time = %5.1f" % machTime

print "lv => lv"
pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(lv_df.head(smpl_num), lv_df.head(500), \
                                                            lv_conf_colns, perf_coln, \
                                                            lv_df.head(500).shape[0] * 0.005)
find_top_rank(pred_top_smpl, lv_df.head(500), lv_conf_colns, perf_coln)
lv_timeout = max(get_vld_df(lv_df)['run_time'].values) * 1.1
machTime = sa_mach_time(lv_df.head(smpl_num), lv_timeout)
print "Machine time = %5.1f" % machTime

print "lvp => lvp"
pred_top_smpl, err_prcntl_df, top_rs_df = whl_pred_top_eval(lv_in_df.head(smpl_num), lv_in_df, \
                                                            lv_in_conf_colns, perf_coln, \
                                                            lv_in_df.shape[0] * 0.005)
find_top_rank(pred_top_smpl, lv_in_df, lv_in_conf_colns, perf_coln)
lv_in_timeout = max(get_vld_df(lv_in_df)['run_time'].values) * 1.1
machTime = sa_mach_time(lv_in_df.head(smpl_num), lv_in_timeout)
print "Machine time = %5.1f" % machTime

In [ ]:
lv_in_df_vld = get_vld_df(lv_in_df)
yp = lv_in_df_vld.run_time.mean()
err  = lv_in_df_vld.run_time.values - yp
rerr = np.abs(err) / lv_in_df_vld.run_time.values * 100
print np.percentile(rerr, 50)

lv_df_vld = get_vld_df(lv_df)
yp = lv_df_vld.run_time.mean()
err  = lv_df_vld.run_time.values - yp
rerr = np.abs(err) / lv_df_vld.run_time.values * 100
print np.percentile(rerr, 50)

lv_df_vld = get_vld_df(lv_df)
yp = lv_df_vld.mach_time.mean()
err  = lv_df_vld.mach_time.values - yp
rerr = np.abs(err) / lv_df_vld.mach_time.values * 100
print np.percentile(rerr, 50)

lv_df_vld = get_vld_df(lv_df)
yp = 0.5 * lv_df_vld.mach_time.mean() + 0.5 * lv_df_vld.run_time.mean()
err  = 0.5 * lv_df_vld.mach_time.values + 0.5 * lv_df_vld.run_time.values - yp
rerr = np.abs(err) / (0.5 * lv_df_vld.mach_time.values + 0.5 * lv_df_vld.run_time.values) * 100
print np.percentile(rerr, 50)

lmp_df_vld = get_vld_df(lmp_df)
yp = lmp_df_vld.run_time.mean()
err  = lmp_df_vld.run_time.values - yp
rerr = np.abs(err) / lmp_df_vld.run_time.values * 100
print np.percentile(rerr, 50)

voro_df_vld = get_vld_df(voro_df)
yp = voro_df_vld.run_time.mean()
err  = voro_df_vld.run_time.values - yp
rerr = np.abs(err) / voro_df_vld.run_time.values * 100
print np.percentile(rerr, 50)

In [ ]:
from scipy.optimize import brute
from Queue import PriorityQueue
import time

def obj_func(x, *params):
    lmp_nproc = round(x[0])
    lmp_ppw = round(x[1])
    lmp_nthread = round(x[2])
    lmp_io_step = round(x[3])
    voro_nproc = round(x[4])
    voro_ppw = round(x[5])
    voro_nthread = round(x[6])
    y = float('inf')
    if (lmp_nproc >= lmp_ppw and voro_nproc >= voro_ppw):
        if (lmp_nproc % lmp_ppw == 0 and voro_nproc % voro_ppw == 0):
            nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw
        elif (lmp_nproc % lmp_ppw == 0 or voro_nproc % voro_ppw == 0):
            nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 1
        else:
            nodes = lmp_nproc // lmp_ppw + voro_nproc // voro_ppw + 2
        if (nodes <= lv_node_num):
            mdl = params[0]
            y = mdl.predict([x])[0]
    return y

def pred_top(xranges, mdl, topn):
    params = (mdl, 0)
    resbrute = brute(obj_func, xranges, args=params, full_output=True, finish=None)
    x0 = resbrute[0]
    fval = resbrute[1]
    grid = resbrute[2]
    Jout = resbrute[3]

    Jout_shape = Jout.shape
    Jout = np.reshape(Jout, \
                      Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                      * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid0 = np.reshape(grid[0], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid1 = np.reshape(grid[1], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid2 = np.reshape(grid[2], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid3 = np.reshape(grid[3], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid4 = np.reshape(grid[4], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid5 = np.reshape(grid[5], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    grid6 = np.reshape(grid[6], \
                       Jout_shape[0] * Jout_shape[1] * Jout_shape[2] * Jout_shape[3] \
                       * Jout_shape[4] * Jout_shape[5] * Jout_shape[6])
    idx = Jout.argsort()[:topn]
    pred_top_df = pd.DataFrame(np.c_[Jout[idx], \
                                     grid0[idx], grid1[idx], grid2[idx], grid3[idx], \
                                     grid4[idx], grid5[idx], grid6[idx]], \
                               columns=['yp_lv', \
                                        'lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step', \
                                        'voro_nproc', 'voro_ppw', 'voro_nthread'])
    return pred_top_df

def process_data(prioQ, mdl, topn):
    xranges = (slice(8.0, (core_num - 1.0) * (lv_node_num - 1.0) + 0.1, 16.0), \
               slice(8.0, core_num, 8.0), \
               slice(1.0, 4.1, 1.0), \
               slice(50.0, 400.1, 50.0), \
               slice(16.0, (core_num - 1.0) * lv_node_num / 2 + 0.1, 16.0), \
               slice(8.0, core_num, 8.0), \
               slice(1.0, 4.1, 1.0))
#    xranges = (slice(8.0 + 4.0 * threadID, (core_num - 1.0) * (lv_node_num - 1.0) + 0.1, 64.0), \
#               slice(4.0, core_num, 4.0), \
#               slice(1.0, 4.1, 1.0), \
#               slice(50.0, 400.1, 50.0), \
#               slice(4.0, (core_num - 1.0) * lv_node_num / 2 + 0.1, 4.0), \
#               slice(4.0, core_num, 4.0), \
#               slice(1.0, 4.1, 1.0))
    pred_top_df = pred_top(xranges, mdl, topn)
    for i in range(pred_top_df.shape[0]):
        prioQ.put(list(pred_top_df.values[i, :]))


def get_top_smpl(mdl, topn):
    startT = time.time()
    
    prioQ = PriorityQueue()
    process_data(prioQ, mdl, topn)
    
    for i in range(topn):
        if (i == 0):
            top_smpl_arr = np.asarray(prioQ.get())
        else:
            top_smpl_arr = np.vstack((top_smpl_arr, np.asarray(prioQ.get())))
    top_smpl_df = pd.DataFrame(np.c_[top_smpl_arr[:, 1:], top_smpl_arr[:, 0]], \
                           columns=['lmp_nproc', 'lmp_ppw', 'lmp_nthread', 'lmp_io_step', \
                                    'voro_nproc', 'voro_ppw', 'voro_nthread', 'yp_lv'])
    
    runtime = time.time() - startT
    print "Execution time is %0.4f s." % runtime
    return top_smpl_df

'''
mdl = train_mdl(lv_df)
topn = 10
top_smpl_df = get_top_smpl(mdl, topn)
print top_smpl_df
df2csv(top_smpl_df, "brute.csv")
'''